In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns',30)
import os

In [2]:
pwd

'C:\\Users\\FN41058\\PYTHON'

In [3]:
os.chdir('C:\\Users\\FN41058\\PYTHON\\Master_Funding')

In [4]:
import os

cwd = os.getcwd()

In [5]:
pwd

'C:\\Users\\FN41058\\PYTHON\\Master_Funding'

## Read Data

In [6]:
# # Current TBL_BAL
# current = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241021.csv', 
# usecols=['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH', 'GCIF_NO', 'CIF_NO',
# 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE', 'CURR_CODE', 'RATE_DPK', 'BASE_AMOUNT', 'NPK_SALES', 'COLT'],
# dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# current = current[~current['SEGMENT'].str.startswith('CFS-RB')].copy()

In [7]:
# usecols=['base_dt','agree_id','flag','acct_no','region','area',
# 'branch','gcif_no','cif_no','sub_prod_nm','segment','gcif_name','prod_type',
# 'curr_code','rate_dpk','base_amount','npk_sales','colt']


In [8]:
# # current = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241105.csv', 
# # usecols=['base_dt','agree_id','flag','acct_no','region','area',
# # 'branch','gcif_no','cif_no','sub_prod_nm','segment','gcif_name','prod_type',
# # 'curr_code','rate_dpk','base_amount','npk_sales','colt'],low_memory=False,dtype={'acct_no':'str', 'cif_no':'str'})

# current = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241107_ori.csv', 
# usecols=['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH', 'GCIF_NO', 'CIF_NO',
# 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE', 'CURR_CODE', 'RATE_DPK', 'BASE_AMOUNT', 'NPK_SALES', 'COLT'],low_memory=False,dtype={'acct_no':'str', 'cif_no':'str'})

# # current.columns = ['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH', 'GCIF_NO', 'CIF_NO',
# # 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE', 'CURR_CODE', 'RATE_DPK', 'BASE_AMOUNT', 'NPK_SALES', 'COLT']

# #current['BASE_AMOUNT'] = pd.to_numeric(current['BASE_AMOUNT'].str.replace(",",".",regex=True))
# #current['RATE_DPK'] = pd.to_numeric(current['RATE_DPK'].str.replace(",",".",regex=True))
# print(current.shape)
# current = current[~current['SEGMENT'].str.startswith('CFS-RB')].copy()
# print(current.shape)


In [9]:
bank_list = pd.read_excel("Bank_Lists.xlsx")
bank_list2 = bank_list['GCIF_NO'].drop_duplicates().tolist()
len(bank_list2)
bank_list.head(3)

,GCIF_NO,CIF_NO,NAMES,Bank
0,G804333276,7907105,BANK PANIN,Bank
1,G403711088,12284319,BANK RESONA PERDANIA PT,Bank
2,G809522766,12593425,BANK WOORI SAUDARA INDONESIA 190,Bank


In [10]:
# Check bila pindah filter from SQL Query to Python
# CURRENT CODE BARU

current = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241129.csv',low_memory=False,dtype={'acct_no':'str', 'cif_no':'str'})
#current = pd.read_excel(cwd+'\\TBL_BAL\\TBL_BAL_241126.xlsx', sheet_name="Export Worksheet")

print(current.shape)
current['SYSTEM_TYPE'] = current['SYSTEM_TYPE'].astype(str)

#extract & append bank cust
bank_cust_curr = current[current['GCIF_NO'].isin(bank_list2)]
bank_cust_curr = bank_cust_curr[bank_cust_curr['SEGMENT']=='XXX-GB-CORP']
print(bank_cust_curr.shape)

current['Filter'] = np.where(
                                (
                                    (current['AGREE_ID'].str.startswith('ST'))
                                    &
                                    (~current['STATUS'].isin([3,4,5]))
                                    &
                                    (current['SYSTEM_TYPE'].str.startswith('8'))
                                    &
                                    (current['CTRL3']!= '797')
                                )
                                |
                                (
                                    (current['AGREE_ID'].str.startswith('IM'))
                                    &
                                    (~current['STATUS'].isin([4,5,8]))
                                    &
                                    (current['SYSTEM_TYPE'].str.startswith('8'))
                                    &
                                    (current['CTRL3']!= '797')
                                )
                                ,"Yes","")

print(current.shape)

current=current[~current['SEGMENT'].isin(['CFS-RB-5MB','CFS-RB-3EA','CFS-RB-1PW'])]
current=current[current['Filter']=="Yes"]
print(current.shape)

#take out yg sudah ada di Prev
print(bank_cust_curr.shape)
bank_cust_curr = bank_cust_curr[~(bank_cust_curr['AGREE_ID'].isin(current['AGREE_ID']))]
print(bank_cust_curr.shape)

current = current.append([bank_cust_curr], ignore_index=True)
print(current.shape)

current.head(3)

(117398, 22)
(217, 22)
(117398, 23)
(81671, 23)
(217, 22)
(201, 22)
(81872, 23)


,BASE_DT,AGREE_ID,FLAG,ACCT_NO,ACCT_BR,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,RATE_DPK,BASE_AMOUNT,NPK_SALES,COLT,STATUS,SYSTEM_TYPE,CTRL3,Filter
0,20241129,IM16IDR0012001074391,Conven,2001074391,1,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735,Maybank Giro,CFS-NONRB-BB,SIBALEC POWEL CABLE & ELECTRICAL SUPPLY,CA,IDR,1.25,6.923490e+08,NaN,NaN,0,870,1,Yes
1,20241129,ST16IDR76730000076708941,Sharia,30000076708941,206,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764,Maybank Deposito iB,CFS-NONRB-BB,"SUMMARECON AGUNG TBK, PT",TD,IDR,2.50,1.250825e+08,0000034089,NaN,0,870,767,Yes
2,20241129,IM16IDR0082008600017,Conven,2008600017,8,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945,Maybank Giro,CFS-NONRB-RSME,TJOKRO BERSAUDARA,CA,IDR,0.00,0.000000e+00,NaN,NaN,6,870,8,Yes


In [11]:
current.drop(['ACCT_BR','STATUS','SYSTEM_TYPE','CTRL3','Filter'], axis=1, inplace=True) 
current.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81872 entries, 0 to 81871
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BASE_DT      81872 non-null  int64  
 1   AGREE_ID     81872 non-null  object 
 2   FLAG         81872 non-null  object 
 3   ACCT_NO      81872 non-null  int64  
 4   REGION       81871 non-null  object 
 5   AREA         81871 non-null  object 
 6   BRANCH       81871 non-null  object 
 7   GCIF_NO      81872 non-null  object 
 8   CIF_NO       81872 non-null  int64  
 9   SUB_PROD_NM  81872 non-null  object 
 10  SEGMENT      81872 non-null  object 
 11  GCIF_NAME    81872 non-null  object 
 12  PROD_TYPE    81872 non-null  object 
 13  CURR_CODE    81872 non-null  object 
 14  RATE_DPK     81872 non-null  float64
 15  BASE_AMOUNT  81872 non-null  float64
 16  NPK_SALES    78038 non-null  object 
 17  COLT         1279 non-null   float64
dtypes: float64(3), int64(3), object(12)
memory usa

In [12]:
# prev D-1 TBL_BAL CODE BARU

prev = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241128.csv',low_memory=False,dtype={'acct_no':'str', 'cif_no':'str'})
#prev = pd.read_excel(cwd+'\\TBL_BAL\\TBL_BAL_241125.xlsx', sheet_name="Export Worksheet")

print(prev.shape)
prev['SYSTEM_TYPE'] = prev['SYSTEM_TYPE'].astype(str)

#extract & append bank cust
bank_cust_prev = prev[prev['GCIF_NO'].isin(bank_list2)]
bank_cust_prev = bank_cust_prev[bank_cust_prev['SEGMENT']=='XXX-GB-CORP']
print(bank_cust_prev.shape)

prev['Filter'] = np.where(
                                (
                                    (prev['AGREE_ID'].str.startswith('ST'))
                                    &
                                    (~prev['STATUS'].isin([3,4,5]))
                                    &
                                    (prev['SYSTEM_TYPE'].str.startswith('8'))
                                    &
                                    (prev['CTRL3']!= '797')
                                )
                                |
                                (
                                    (prev['AGREE_ID'].str.startswith('IM'))
                                    &
                                    (~prev['STATUS'].isin([4,5,8]))
                                    &
                                    (prev['SYSTEM_TYPE'].str.startswith('8'))
                                    &
                                    (prev['CTRL3']!= '797')
                                )
                                ,"Yes","")

print(prev.shape)

prev=prev[~prev['SEGMENT'].isin(['CFS-RB-5MB'])]
prev=prev[prev['Filter']=="Yes"]
print(prev.shape)

#take out yg sudah ada di Prev
print(bank_cust_prev.shape)
bank_cust_prev = bank_cust_prev[~(bank_cust_prev['AGREE_ID'].isin(prev['AGREE_ID']))]
print(bank_cust_prev.shape)

prev = prev.append([bank_cust_prev], ignore_index=True)
print(prev.shape)

prev=prev[['BASE_DT','AGREE_ID', 'BASE_AMOUNT', 'SEGMENT']]
prev = prev[['AGREE_ID', 'BASE_AMOUNT']].copy()
prev.head(3)

(117266, 22)
(217, 22)
(117266, 23)
(81664, 23)
(217, 22)
(201, 22)
(81865, 23)


,AGREE_ID,BASE_AMOUNT
0,ST16IDR59000001590999900,0.0
1,ST16IDR00130020600113766,38369863.0
2,ST16IDR76730000076708767,124308566.0


In [13]:
#prev.to_csv('tbl_bal_241108_filtered.csv')

In [14]:
# # prev D-1 TBL_BAL
# prev = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241107.csv',
# usecols=['BASE_DT','AGREE_ID', 'BASE_AMOUNT', 'SEGMENT'],
# dtype={'AGREE_ID':'str','ACCT_NO':'str', 'CIF_NO':'str', 'COLT':'float'},low_memory=False)
# #prev['BASE_AMOUNT'] = pd.to_numeric(prev['BASE_AMOUNT'].str.replace(",",".",regex=True))
# prev = prev[~prev['SEGMENT'].str.startswith('CFS-RB')].copy()
# #prev = prev[prev['BASE_DT']==20241030]
# prev = prev[['AGREE_ID', 'BASE_AMOUNT']].copy()
# prev.head(3)

In [15]:
# M-1 TBL BAL
prevMonth = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241031.csv', 
usecols=['AGREE_ID', 'BASE_AMOUNT', 'SEGMENT'],
dtype={'ACCT_NO':'str', 'CIF_NO':'str'})
prevMonth = prevMonth[~prevMonth['SEGMENT'].str.startswith('CFS-RB')].copy()
prevMonth = prevMonth[['AGREE_ID', 'BASE_AMOUNT']].copy()
prevMonth.head(3)

,AGREE_ID,BASE_AMOUNT
0,IM16IDR7108710006982,0.00
1,IM16USD1672167001720,13464583.55
2,IM16IDR7108710224150,0.00


In [16]:
# End of last year TBL BAL
prevYr = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_231229.csv', 
encoding='windows-1252',usecols=['AGREE_ID', 'BASE_AMOUNT', 'SEGMENT'],
dtype={'ACCT_NO':'str', 'CIF_NO':'str'})
prevYr = prevYr[~prevYr['SEGMENT'].str.startswith('CFS-RB')].copy()
prevYr = prevYr[['AGREE_ID', 'BASE_AMOUNT']].copy()
prevYr.head(3)

,AGREE_ID,BASE_AMOUNT
0,ST16IDR01630000001616790,1.980000e+10
1,IM16IDR7718771900812,2.427617e+07
2,IM16IDR1532153352802,-1.416179e+09


In [17]:
# RMR Data
RMR = pd.read_excel(cwd+'\\RMR\\RMR 2410.xlsx')
RMR.head(3)

,BASE_DT,FLAG,SEGMENT,PROD_TYPE,BASE_AMT_FIX,Unnamed: 5,Unnamed: 6
0,20241031,Conven,CFS-NONRB-BB,CA,6.208786e+12,NaN,NaN
1,20241031,Conven,CFS-NONRB-BB,SA,6.493364e+10,NaN,NaN
2,20241031,Conven,CFS-NONRB-BB,TD,6.075005e+12,NaN,NaN


In [18]:
# Target Data
TARGET = pd.read_excel(cwd+'\\Target\\Target 2411.xlsx')
TARGET.head(3)

,BASE_DT,FLAG,SEGMENT_FIX,PROD_TYPE,BASE_AMT_FIX,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,20241110,Conven,CFS-BB,CA,1.055089e+13,NaN,NaN,NaN,NaN
1,20241110,Conven,CFS-BB,SA,7.279970e+10,NaN,NaN,NaN,NaN
2,20241110,Conven,CFS-BB,TD,6.558724e+12,NaN,NaN,NaN,NaN


In [19]:
# Funding Flag
FFlag = pd.read_csv(cwd+'\\FLeakagePot\\FLeakagePot_2409.csv')
FFlag = FFlag[['GCIF','FINAL_FLAG']]
FFlag.head(3)

,GCIF,FINAL_FLAG
0,G000022780,DORMANT
1,G000129627,LOW
2,G000210070,LOW


In [20]:
# Division Mapping
divMap = pd.read_csv(cwd+'\\COR & CB\\Division Mapping 2406.csv')
divMap.head(3)

,GCIF_NO,DIVISION
0,G007330509,SOE
1,G403058233,SOE
2,G406923219,SOE


In [21]:
current_d2  ="2024-11-29"
# Change final name heress
finalName = 'tbl_bal_241129_fin.csv'


In [22]:
# READ for MTD COF

column_used_lower=['base_dt','agree_id','flag','acct_no','region','area','branch','gcif_no','cif_no',
'sub_prod_nm','segment','gcif_name','prod_type','curr_code','rate_dpk','base_amount','npk_sales','colt']

column_used = ['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH', 'GCIF_NO', 'CIF_NO',
'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE', 'CURR_CODE', 'RATE_DPK', 'BASE_AMOUNT', 'NPK_SALES', 'COLT']

# MTD TBL BAL


tbl_bal_241101 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241101.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241104 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241104.csv', usecols=column_used_lower, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241105 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241105.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241106 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241106.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241107 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241107.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241108 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241108.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241111 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241111.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241112 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241112.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241113 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241113.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241114 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241114.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241115 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241115.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241118 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241118.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241119 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241119.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241120 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241120.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241121 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241121.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241122 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241122.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241125 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241125.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241126 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241126.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241128 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241128.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
tbl_bal_241129 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241129.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)



#tbl_bal_241122 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241126.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
#tbl_bal_241122 = pd.read_excel(cwd+'\\TBL_BAL\\TBL_BAL_241122.xlsx', sheet_name="Export Worksheet", usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'})
#tbl_bal_241125 = pd.read_excel(cwd+'\\TBL_BAL\\TBL_BAL_241125.xlsx', sheet_name="Export Worksheet", usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'})
#tbl_bal_241126 = pd.read_excel(cwd+'\\TBL_BAL\\TBL_BAL_241126.xlsx', sheet_name="Export Worksheet", usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'})


#convert lower column name to big
tbl_bal_241104.columns = ['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH', 'GCIF_NO', 'CIF_NO',
'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE', 'CURR_CODE', 'RATE_DPK', 'BASE_AMOUNT', 'NPK_SALES', 'COLT']


mtd_tbl_bal = tbl_bal_241101.append([tbl_bal_241104,tbl_bal_241105,tbl_bal_241106,tbl_bal_241107,
                                     tbl_bal_241108,tbl_bal_241111,tbl_bal_241112,tbl_bal_241113,
                                     tbl_bal_241114,tbl_bal_241115,tbl_bal_241118,tbl_bal_241119,
                                     tbl_bal_241120,tbl_bal_241121,tbl_bal_241122,tbl_bal_241125,
                                     tbl_bal_241126,tbl_bal_241128,tbl_bal_241129], ignore_index=True)

print(mtd_tbl_bal.shape)
mtd_tbl_bal = mtd_tbl_bal[~mtd_tbl_bal['SEGMENT'].str.startswith('CFS-RB')].copy()
print(mtd_tbl_bal.shape)

(3348220, 18)
(2240243, 18)


In [23]:
#mtd_tbl_bal.to_csv('mtd_tbl_bal_test.csv', index=False)

In [24]:
# # READ for MTD COF
# column_used = ['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH', 'GCIF_NO', 'CIF_NO',
# 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE', 'CURR_CODE', 'RATE_DPK', 'BASE_AMOUNT', 'NPK_SALES', 'COLT']

# # MTD TBL BAL
# tbl_bal_240902 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240902.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240903 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240903.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240904 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240904.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240905 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240905.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240906 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240906.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240909 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240909.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240910 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240910.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240911 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240911.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240912 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240912.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240913 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240913.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240917 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240917.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240918 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240918.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240919 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240919.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240920 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240920.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240923 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240923.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240924 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240924.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240925 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240925.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240926 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240926.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240927 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240927.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_240930 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_240930.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241001 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241001.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241002 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241002.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241003 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241003.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241004 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241004.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241007 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241007.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241008 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241008.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241009 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241009.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241010 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241010.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241011 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241011.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241014 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241014.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241015 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241015.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241016 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241016.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241017 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241017.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241018 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241018.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal_241021 = pd.read_csv(cwd+'\\TBL_BAL\\tbl_bal_241021.csv', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str','BASE_AMOUNT':'float'},low_memory=False)
# tbl_bal = pd.read_csv(cwd+'\\TBL_BAL\\MASTER_FUNDING_OCT.dsv', sep='|', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str'},low_memory=False)
# tbl_bal['BASE_AMOUNT'] = pd.to_numeric(tbl_bal['BASE_AMOUNT'].str.replace(",",".",regex=True))
# tbl_bal['RATE_DPK'] = pd.to_numeric(tbl_bal['RATE_DPK'].str.replace(",",".",regex=True))
# tbl_bal_241022 = tbl_bal[tbl_bal['BASE_DT']==20241022]
# tbl_bal_241023 = tbl_bal[tbl_bal['BASE_DT']==20241023]
# tbl_bal_241024 = tbl_bal[tbl_bal['BASE_DT']==20241024]
# tbl_bal_241025 = tbl_bal[tbl_bal['BASE_DT']==20241025]
# tbl_bal2 = pd.read_csv(cwd+'\\TBL_BAL\\MASTER_FUNDING_NOV.dsv', sep='|', usecols=column_used, dtype={'ACCT_NO':'str', 'CIF_NO':'str'},low_memory=False)
# tbl_bal2['BASE_AMOUNT'] = pd.to_numeric(tbl_bal2['BASE_AMOUNT'].str.replace(",",".",regex=True))
# tbl_bal2['RATE_DPK'] = pd.to_numeric(tbl_bal2['RATE_DPK'].str.replace(",",".",regex=True))
# tbl_bal_241028 = tbl_bal2[tbl_bal2['BASE_DT']==20241028]
# tbl_bal_241029 = tbl_bal2[tbl_bal2['BASE_DT']==20241029]
# tbl_bal_241030 = tbl_bal2[tbl_bal2['BASE_DT']==20241030]
# tbl_bal_241031 = tbl_bal2[tbl_bal2['BASE_DT']==20241031]

# mtd_tbl_bal = tbl_bal_240902.append([tbl_bal_240903,tbl_bal_240904,tbl_bal_240905,tbl_bal_240906,
#                                      tbl_bal_240909, tbl_bal_240910, tbl_bal_240911, tbl_bal_240912, tbl_bal_240913, tbl_bal_240917,
#                                      tbl_bal_240918, tbl_bal_240919, tbl_bal_240920, tbl_bal_240923, tbl_bal_240924,
#                                      tbl_bal_240925, tbl_bal_240926, tbl_bal_240927, tbl_bal_240930, tbl_bal_241001, tbl_bal_241002, tbl_bal_241003, 
#                                      tbl_bal_241004, tbl_bal_241007, tbl_bal_241008, tbl_bal_241009, tbl_bal_241010, tbl_bal_241011, tbl_bal_241014, tbl_bal_241015,
#                                      tbl_bal_241016, tbl_bal_241017, tbl_bal_240918, tbl_bal_241021, tbl_bal_241022, tbl_bal_241023, tbl_bal_241024, tbl_bal_241025,
#                                      tbl_bal_241028, tbl_bal_241029, tbl_bal_241030, tbl_bal_241031], ignore_index=True)

# print(mtd_tbl_bal.shape)
# mtd_tbl_bal = mtd_tbl_bal[~mtd_tbl_bal['SEGMENT'].str.startswith('CFS-RB')].copy()
# print(mtd_tbl_bal.shape)

## Calculate DtD

In [25]:
current.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81872 entries, 0 to 81871
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BASE_DT      81872 non-null  int64  
 1   AGREE_ID     81872 non-null  object 
 2   FLAG         81872 non-null  object 
 3   ACCT_NO      81872 non-null  int64  
 4   REGION       81871 non-null  object 
 5   AREA         81871 non-null  object 
 6   BRANCH       81871 non-null  object 
 7   GCIF_NO      81872 non-null  object 
 8   CIF_NO       81872 non-null  int64  
 9   SUB_PROD_NM  81872 non-null  object 
 10  SEGMENT      81872 non-null  object 
 11  GCIF_NAME    81872 non-null  object 
 12  PROD_TYPE    81872 non-null  object 
 13  CURR_CODE    81872 non-null  object 
 14  RATE_DPK     81872 non-null  float64
 15  BASE_AMOUNT  81872 non-null  float64
 16  NPK_SALES    78038 non-null  object 
 17  COLT         1279 non-null   float64
dtypes: float64(3), int64(3), object(12)
memory usa

In [26]:
# Fix BASE_AMOUNT, zeroing negative values
current[['BASE_AMT_FIX']] = current[['BASE_AMOUNT']]
current.loc[current["BASE_AMT_FIX"] < 0, "BASE_AMT_FIX"] = 0

In [27]:
# Compare values
current[['BASE_AMOUNT', 'BASE_AMT_FIX']].describe()

,BASE_AMOUNT,BASE_AMT_FIX
count,8.187200e+04,8.187200e+04
mean,6.971615e+08,9.080836e+08
std,8.830958e+09,8.607563e+09
min,-1.699557e+11,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,3.673255e+06,3.673255e+06
75%,8.324022e+07,8.324022e+07
max,9.823554e+11,9.823554e+11


In [28]:
current.head()

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,RATE_DPK,BASE_AMOUNT,NPK_SALES,COLT,BASE_AMT_FIX
0,20241129,IM16IDR0012001074391,Conven,2001074391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735,Maybank Giro,CFS-NONRB-BB,SIBALEC POWEL CABLE & ELECTRICAL SUPPLY,CA,IDR,1.25,6.923490e+08,NaN,NaN,6.923490e+08
1,20241129,ST16IDR76730000076708941,Sharia,30000076708941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764,Maybank Deposito iB,CFS-NONRB-BB,"SUMMARECON AGUNG TBK, PT",TD,IDR,2.50,1.250825e+08,0000034089,NaN,1.250825e+08
2,20241129,IM16IDR0082008600017,Conven,2008600017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945,Maybank Giro,CFS-NONRB-RSME,TJOKRO BERSAUDARA,CA,IDR,0.00,0.000000e+00,NaN,NaN,0.000000e+00
3,20241129,IM16IDR0202020901674,Conven,2020901674,JAKARTA 1,FATMAWATI,KCP AMPERA,G000458489,108120,Maybank Giro,CFS-NONRB-RSME,"WADHA ARTHA ABADI, PT",CA,IDR,0.00,0.000000e+00,NaN,NaN,0.000000e+00
4,20241129,IM16USD1212121101114,Conven,2121101114,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G000473942,870324,Maybank Giro,CFS-NONRB-RSME,"BADE MAKMUR ORISSA, PT",CA,USD,0.00,4.801098e+07,0000019696,NaN,4.801098e+07


In [29]:
prev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81865 entries, 0 to 81864
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AGREE_ID     81865 non-null  object 
 1   BASE_AMOUNT  81865 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.2+ MB


In [30]:
# Fix BASE_AMOUNT, zeroing negative values
prev[['BASE_AMT_FIX']] = prev[['BASE_AMOUNT']]
prev.loc[prev["BASE_AMT_FIX"] < 0, "BASE_AMT_FIX"] = 0

In [31]:
prev

,AGREE_ID,BASE_AMOUNT,BASE_AMT_FIX
0,ST16IDR59000001590999900,0.000000e+00,0.000000e+00
1,ST16IDR00130020600113766,3.836986e+07,3.836986e+07
2,ST16IDR76730000076708767,1.243086e+08,1.243086e+08
3,IM16IDR0162016301778,1.789544e+09,1.789544e+09
4,ST16IDR13230000013204256,1.357466e+08,1.357466e+08
...,...,...,...
81860,IM16IDR0062006001658,0.000000e+00,0.000000e+00
81861,IM16IDR0062006111838,0.000000e+00,0.000000e+00
81862,IM16IDR4632463000028,0.000000e+00,0.000000e+00
81863,IM16IDR5962596000875,3.368003e+08,3.368003e+08


In [32]:
# Take necessary column, rename
prev = prev[['AGREE_ID', 'BASE_AMT_FIX']]
prev.columns = ['AGREE_ID', 'PREV_AMT_FIX']

In [33]:
# Merge D and D-1 data
print(current.shape)
currFin = current.merge(prev, on='AGREE_ID', how='left')
print(currFin.shape)

(81872, 19)
(81872, 20)


In [34]:
currFin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81872 entries, 0 to 81871
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BASE_DT       81872 non-null  int64  
 1   AGREE_ID      81872 non-null  object 
 2   FLAG          81872 non-null  object 
 3   ACCT_NO       81872 non-null  int64  
 4   REGION        81871 non-null  object 
 5   AREA          81871 non-null  object 
 6   BRANCH        81871 non-null  object 
 7   GCIF_NO       81872 non-null  object 
 8   CIF_NO        81872 non-null  int64  
 9   SUB_PROD_NM   81872 non-null  object 
 10  SEGMENT       81872 non-null  object 
 11  GCIF_NAME     81872 non-null  object 
 12  PROD_TYPE     81872 non-null  object 
 13  CURR_CODE     81872 non-null  object 
 14  RATE_DPK      81872 non-null  float64
 15  BASE_AMOUNT   81872 non-null  float64
 16  NPK_SALES     78038 non-null  object 
 17  COLT          1279 non-null   float64
 18  BASE_AMT_FIX  81872 non-nu

In [35]:
# Filk NaN caused by joining
currFin[['PREV_AMT_FIX']] = currFin[['PREV_AMT_FIX']].fillna(value=0)

In [36]:
# Calculate DtD
currFin['DtD'] = currFin['BASE_AMT_FIX'] - currFin['PREV_AMT_FIX']

In [37]:
currFin

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,RATE_DPK,BASE_AMOUNT,NPK_SALES,COLT,BASE_AMT_FIX,PREV_AMT_FIX,DtD
0,20241129,IM16IDR0012001074391,Conven,2001074391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735,Maybank Giro,CFS-NONRB-BB,SIBALEC POWEL CABLE & ELECTRICAL SUPPLY,CA,IDR,1.25,6.923490e+08,NaN,NaN,6.923490e+08,6.919798e+08,3.691690e+05
1,20241129,ST16IDR76730000076708941,Sharia,30000076708941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764,Maybank Deposito iB,CFS-NONRB-BB,"SUMMARECON AGUNG TBK, PT",TD,IDR,2.50,1.250825e+08,0000034089,NaN,1.250825e+08,1.250825e+08,0.000000e+00
2,20241129,IM16IDR0082008600017,Conven,2008600017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945,Maybank Giro,CFS-NONRB-RSME,TJOKRO BERSAUDARA,CA,IDR,0.00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00
3,20241129,IM16IDR0202020901674,Conven,2020901674,JAKARTA 1,FATMAWATI,KCP AMPERA,G000458489,108120,Maybank Giro,CFS-NONRB-RSME,"WADHA ARTHA ABADI, PT",CA,IDR,0.00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00
4,20241129,IM16USD1212121101114,Conven,2121101114,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G000473942,870324,Maybank Giro,CFS-NONRB-RSME,"BADE MAKMUR ORISSA, PT",CA,USD,0.00,4.801098e+07,0000019696,NaN,4.801098e+07,4.808673e+07,-7.575100e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81867,20241129,IM16IDR4632463000019,Conven,2463000019,SULAWESI & IBT,SULAWESI UTARA & IBT,KCI TERNATE,G702243825,3590560,Maybank Giro,XXX-GB-CORP,BPR MODERN EXPRESS,CA,IDR,0.25,7.640283e+07,0000018734,NaN,7.640283e+07,7.638819e+07,1.464100e+04
81868,20241129,IM16IDR0652065000091,Conven,2065000091,SULAWESI & IBT,SULAWESI UTARA & IBT,KCI AMBON,G702243825,3590560,Maybank Giro,XXX-GB-CORP,BPR MODERN EXPRESS,CA,IDR,4.25,6.767913e+10,0000018734,NaN,6.767913e+10,6.617644e+10,1.502686e+09
81869,20241129,IM16IDR2882288001384,Conven,2288001384,JAKARTA 2,PURI KENCANA,KCI PURI KENCANA,G705751519,9162592,Maybank Giro,XXX-GB-CORP,BANK TABUNGAN NEGARA PERSERO TBK PT,CA,IDR,1.00,1.005683e+11,0000023992,NaN,1.005683e+11,1.004861e+11,8.224912e+07
81870,20241129,IM16IDR5962596000840,Conven,2596000840,JAKARTA 1,SUMMITMAS,KCP SS3,G710119561,13199210,Maybank Giro,XXX-GB-CORP,BPR BERKAH PERSERODA PT,CA,IDR,3.00,1.548047e+09,0000036154,NaN,1.548047e+09,1.544239e+09,3.807693e+06


In [38]:
currFin.columns

Index(['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH',
       'GCIF_NO', 'CIF_NO', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE',
       'CURR_CODE', 'RATE_DPK', 'BASE_AMOUNT', 'NPK_SALES', 'COLT',
       'BASE_AMT_FIX', 'PREV_AMT_FIX', 'DtD'],
      dtype='object')

In [39]:
currFin = currFin[['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH',
       'GCIF_NO', 'CIF_NO', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE',
       'CURR_CODE', 'NPK_SALES','COLT','RATE_DPK', 'BASE_AMOUNT', 'BASE_AMT_FIX',
       'DtD']]

# Calculate MtD

In [40]:
# Fix BASE_AMOUNT, zeroing negative values
prevMonth[['BASE_AMT_FIX']] = prevMonth[['BASE_AMOUNT']]
prevMonth.loc[prevMonth["BASE_AMT_FIX"] < 0, "BASE_AMT_FIX"] = 0

In [41]:
prevMonth.head()

,AGREE_ID,BASE_AMOUNT,BASE_AMT_FIX
0,IM16IDR7108710006982,0.000000e+00,0.000000e+00
1,IM16USD1672167001720,1.346458e+07,1.346458e+07
2,IM16IDR7108710224150,0.000000e+00,0.000000e+00
3,IM16IDR0772077360592,3.112121e+08,3.112121e+08
4,IM16IDR0202020901674,0.000000e+00,0.000000e+00


In [42]:
# Take necessary columns, rename
prevMonth = prevMonth[['AGREE_ID', 'BASE_AMT_FIX']]
prevMonth.columns = ['AGREE_ID', 'PREVMONTH_AMT_FIX']

In [43]:
# Join D and M-1 data
print(currFin.shape)
currFin = currFin.merge(prevMonth, on='AGREE_ID', how='left')
print(currFin.shape)

(81872, 20)
(81872, 21)


In [44]:
currFin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81872 entries, 0 to 81871
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   BASE_DT            81872 non-null  int64  
 1   AGREE_ID           81872 non-null  object 
 2   FLAG               81872 non-null  object 
 3   ACCT_NO            81872 non-null  int64  
 4   REGION             81871 non-null  object 
 5   AREA               81871 non-null  object 
 6   BRANCH             81871 non-null  object 
 7   GCIF_NO            81872 non-null  object 
 8   CIF_NO             81872 non-null  int64  
 9   SUB_PROD_NM        81872 non-null  object 
 10  SEGMENT            81872 non-null  object 
 11  GCIF_NAME          81872 non-null  object 
 12  PROD_TYPE          81872 non-null  object 
 13  CURR_CODE          81872 non-null  object 
 14  NPK_SALES          78038 non-null  object 
 15  COLT               1279 non-null   float64
 16  RATE_DPK           818

In [45]:
# Fill NaN caused by joining
currFin[['PREVMONTH_AMT_FIX']] = currFin[['PREVMONTH_AMT_FIX']].fillna(value=0)

In [46]:
# Calculate MtD
currFin['MtD'] = currFin['BASE_AMT_FIX'] - currFin['PREVMONTH_AMT_FIX']

In [47]:
currFin

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,COLT,RATE_DPK,BASE_AMOUNT,BASE_AMT_FIX,DtD,PREVMONTH_AMT_FIX,MtD
0,20241129,IM16IDR0012001074391,Conven,2001074391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735,Maybank Giro,CFS-NONRB-BB,SIBALEC POWEL CABLE & ELECTRICAL SUPPLY,CA,IDR,NaN,NaN,1.25,6.923490e+08,6.923490e+08,3.691690e+05,6.784458e+08,13903205.00
1,20241129,ST16IDR76730000076708941,Sharia,30000076708941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764,Maybank Deposito iB,CFS-NONRB-BB,"SUMMARECON AGUNG TBK, PT",TD,IDR,0000034089,NaN,2.50,1.250825e+08,1.250825e+08,0.000000e+00,1.250825e+08,0.00
2,20241129,IM16IDR0082008600017,Conven,2008600017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945,Maybank Giro,CFS-NONRB-RSME,TJOKRO BERSAUDARA,CA,IDR,NaN,NaN,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00
3,20241129,IM16IDR0202020901674,Conven,2020901674,JAKARTA 1,FATMAWATI,KCP AMPERA,G000458489,108120,Maybank Giro,CFS-NONRB-RSME,"WADHA ARTHA ABADI, PT",CA,IDR,NaN,NaN,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00
4,20241129,IM16USD1212121101114,Conven,2121101114,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G000473942,870324,Maybank Giro,CFS-NONRB-RSME,"BADE MAKMUR ORISSA, PT",CA,USD,0000019696,NaN,0.00,4.801098e+07,4.801098e+07,-7.575100e+04,4.763495e+07,376031.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81867,20241129,IM16IDR4632463000019,Conven,2463000019,SULAWESI & IBT,SULAWESI UTARA & IBT,KCI TERNATE,G702243825,3590560,Maybank Giro,XXX-GB-CORP,BPR MODERN EXPRESS,CA,IDR,0000018734,NaN,0.25,7.640283e+07,7.640283e+07,1.464100e+04,7.642319e+07,-20359.00
81868,20241129,IM16IDR0652065000091,Conven,2065000091,SULAWESI & IBT,SULAWESI UTARA & IBT,KCI AMBON,G702243825,3590560,Maybank Giro,XXX-GB-CORP,BPR MODERN EXPRESS,CA,IDR,0000018734,NaN,4.25,6.767913e+10,6.767913e+10,1.502686e+09,6.773499e+10,-55859380.68
81869,20241129,IM16IDR2882288001384,Conven,2288001384,JAKARTA 2,PURI KENCANA,KCI PURI KENCANA,G705751519,9162592,Maybank Giro,XXX-GB-CORP,BANK TABUNGAN NEGARA PERSERO TBK PT,CA,IDR,0000023992,NaN,1.00,1.005683e+11,1.005683e+11,8.224912e+07,1.004861e+11,82229124.00
81870,20241129,IM16IDR5962596000840,Conven,2596000840,JAKARTA 1,SUMMITMAS,KCP SS3,G710119561,13199210,Maybank Giro,XXX-GB-CORP,BPR BERKAH PERSERODA PT,CA,IDR,0000036154,NaN,3.00,1.548047e+09,1.548047e+09,3.807693e+06,1.544274e+09,3772693.00


In [48]:
currFin = currFin[['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH',
       'GCIF_NO', 'CIF_NO', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE',
       'CURR_CODE', 'NPK_SALES','COLT','RATE_DPK', 'BASE_AMOUNT', 'BASE_AMT_FIX',
       'DtD','MtD']]

## Calculate YtD

In [49]:
# Fix BASE_AMOUNT, zeroing negative values
prevYr[['BASE_AMT_FIX']] = prevYr[['BASE_AMOUNT']]
prevYr.loc[prevYr["BASE_AMT_FIX"] < 0, "BASE_AMT_FIX"] = 0

In [50]:
prevYr

,AGREE_ID,BASE_AMOUNT,BASE_AMT_FIX
0,ST16IDR01630000001616790,1.980000e+10,1.980000e+10
1,IM16IDR7718771900812,2.427617e+07,2.427617e+07
2,IM16IDR1532153352802,-1.416179e+09,0.000000e+00
3,ST16IDR15830000015805137,2.508363e+08,2.508363e+08
4,IM16IDR2552255005058,1.605675e+07,1.605675e+07
...,...,...,...
88256,ST16IDR14430000014402658,2.500000e+09,2.500000e+09
88257,IM16IDR7872787001027,1.096774e+07,1.096774e+07
88258,IM16CNY0502050000190,0.000000e+00,0.000000e+00
88259,IM16IDR5842584000084,-9.597058e+09,0.000000e+00


In [51]:
# Take necessary columns, rename
prevYr = prevYr[['AGREE_ID', 'BASE_AMT_FIX']]
prevYr.columns = ['AGREE_ID', 'PREVYR_AMT_FIX']

In [52]:
# Join D and M-1 data
print(currFin.shape)
currFin = currFin.merge(prevYr, on='AGREE_ID', how='left')
print(currFin.shape)

(81872, 21)
(81872, 22)


In [53]:
currFin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81872 entries, 0 to 81871
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   BASE_DT         81872 non-null  int64  
 1   AGREE_ID        81872 non-null  object 
 2   FLAG            81872 non-null  object 
 3   ACCT_NO         81872 non-null  int64  
 4   REGION          81871 non-null  object 
 5   AREA            81871 non-null  object 
 6   BRANCH          81871 non-null  object 
 7   GCIF_NO         81872 non-null  object 
 8   CIF_NO          81872 non-null  int64  
 9   SUB_PROD_NM     81872 non-null  object 
 10  SEGMENT         81872 non-null  object 
 11  GCIF_NAME       81872 non-null  object 
 12  PROD_TYPE       81872 non-null  object 
 13  CURR_CODE       81872 non-null  object 
 14  NPK_SALES       78038 non-null  object 
 15  COLT            1279 non-null   float64
 16  RATE_DPK        81872 non-null  float64
 17  BASE_AMOUNT     81872 non-null 

In [54]:
# Fill NaN caused by joining
currFin[['PREVYR_AMT_FIX']] = currFin[['PREVYR_AMT_FIX']].fillna(value=0)

In [55]:
# Calculate YtD
currFin['YtD'] = currFin['BASE_AMT_FIX'] - currFin['PREVYR_AMT_FIX']

In [56]:
currFin

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,COLT,RATE_DPK,BASE_AMOUNT,BASE_AMT_FIX,DtD,MtD,PREVYR_AMT_FIX,YtD
0,20241129,IM16IDR0012001074391,Conven,2001074391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735,Maybank Giro,CFS-NONRB-BB,SIBALEC POWEL CABLE & ELECTRICAL SUPPLY,CA,IDR,NaN,NaN,1.25,6.923490e+08,6.923490e+08,3.691690e+05,13903205.00,5.504947e+08,1.418543e+08
1,20241129,ST16IDR76730000076708941,Sharia,30000076708941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764,Maybank Deposito iB,CFS-NONRB-BB,"SUMMARECON AGUNG TBK, PT",TD,IDR,0000034089,NaN,2.50,1.250825e+08,1.250825e+08,0.000000e+00,0.00,1.250825e+08,0.000000e+00
2,20241129,IM16IDR0082008600017,Conven,2008600017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945,Maybank Giro,CFS-NONRB-RSME,TJOKRO BERSAUDARA,CA,IDR,NaN,NaN,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000e+00
3,20241129,IM16IDR0202020901674,Conven,2020901674,JAKARTA 1,FATMAWATI,KCP AMPERA,G000458489,108120,Maybank Giro,CFS-NONRB-RSME,"WADHA ARTHA ABADI, PT",CA,IDR,NaN,NaN,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000e+00
4,20241129,IM16USD1212121101114,Conven,2121101114,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G000473942,870324,Maybank Giro,CFS-NONRB-RSME,"BADE MAKMUR ORISSA, PT",CA,USD,0000019696,NaN,0.00,4.801098e+07,4.801098e+07,-7.575100e+04,376031.00,4.750036e+07,5.106229e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81867,20241129,IM16IDR4632463000019,Conven,2463000019,SULAWESI & IBT,SULAWESI UTARA & IBT,KCI TERNATE,G702243825,3590560,Maybank Giro,XXX-GB-CORP,BPR MODERN EXPRESS,CA,IDR,0000018734,NaN,0.25,7.640283e+07,7.640283e+07,1.464100e+04,-20359.00,0.000000e+00,7.640283e+07
81868,20241129,IM16IDR0652065000091,Conven,2065000091,SULAWESI & IBT,SULAWESI UTARA & IBT,KCI AMBON,G702243825,3590560,Maybank Giro,XXX-GB-CORP,BPR MODERN EXPRESS,CA,IDR,0000018734,NaN,4.25,6.767913e+10,6.767913e+10,1.502686e+09,-55859380.68,0.000000e+00,6.767913e+10
81869,20241129,IM16IDR2882288001384,Conven,2288001384,JAKARTA 2,PURI KENCANA,KCI PURI KENCANA,G705751519,9162592,Maybank Giro,XXX-GB-CORP,BANK TABUNGAN NEGARA PERSERO TBK PT,CA,IDR,0000023992,NaN,1.00,1.005683e+11,1.005683e+11,8.224912e+07,82229124.00,0.000000e+00,1.005683e+11
81870,20241129,IM16IDR5962596000840,Conven,2596000840,JAKARTA 1,SUMMITMAS,KCP SS3,G710119561,13199210,Maybank Giro,XXX-GB-CORP,BPR BERKAH PERSERODA PT,CA,IDR,0000036154,NaN,3.00,1.548047e+09,1.548047e+09,3.807693e+06,3772693.00,0.000000e+00,1.548047e+09


In [57]:
#adjust ACCT_NO
currFin['ACCT_NO'] = currFin['ACCT_NO'].astype(str)
currFin['ACCT_NO']

0            2001074391
1        30000076708941
2            2008600017
3            2020901674
4            2121101114
              ...      
81867        2463000019
81868        2065000091
81869        2288001384
81870        2596000840
81871        2288000127
Name: ACCT_NO, Length: 81872, dtype: object

In [58]:
# currFin['ACCT_NO'] = currFin['ACCT_NO'].str[:-2]
# currFin['ACCT_NO']

In [59]:
currFin = currFin[['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH',
       'GCIF_NO', 'CIF_NO', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE',
       'CURR_CODE', 'NPK_SALES','COLT','RATE_DPK', 'BASE_AMOUNT', 'BASE_AMT_FIX',
       'DtD','MtD','YtD']]

# Add Division Mapping

In [60]:
divMap

,GCIF_NO,DIVISION
0,G007330509,SOE
1,G403058233,SOE
2,G406923219,SOE
3,G605050137,LLC & MNC
4,G106684566,LLC & MNC
...,...,...
6104,100000165,Mumbai
6105,MUFG,BANK
6106,G906829916,BANK
6107,G009112988,LLC & MNC


In [61]:
# Take necessary columns
divMap = divMap[['GCIF_NO', 'DIVISION']]

In [62]:
divMap = divMap.drop_duplicates('GCIF_NO',keep='first')

In [63]:
divMap

,GCIF_NO,DIVISION
0,G007330509,SOE
1,G403058233,SOE
2,G406923219,SOE
3,G605050137,LLC & MNC
4,G106684566,LLC & MNC
...,...,...
6104,100000165,Mumbai
6105,MUFG,BANK
6106,G906829916,BANK
6107,G009112988,LLC & MNC


In [64]:
# Join division mapping
print(currFin.shape)
currFin = currFin.merge(divMap, on='GCIF_NO', how='left')
print(currFin.shape)

(81872, 22)
(81872, 23)


In [65]:
# Null Division if LoB != GB
currFin.loc[currFin["SEGMENT"].str.contains(pat="CFS", na=False), "DIVISION"] = np.NaN

In [66]:
currFin[currFin['GCIF_NO']=='G302105515']

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,COLT,RATE_DPK,BASE_AMOUNT,BASE_AMT_FIX,DtD,MtD,YtD,DIVISION
42555,20241129,IM16IDR1212121712442,Conven,2121712442,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G302105515,6038286,Maybank Giro,CFS-NONRB-RSME,IWK THAILAND LIMITED,CA,IDR,0000019696,NaN,0.75,2.531914e+08,2.531914e+08,-47530287.0,-74818713.0,156971189.0,NaN


In [67]:
print(currFin.shape)
currFin.head()

(81872, 23)


,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,COLT,RATE_DPK,BASE_AMOUNT,BASE_AMT_FIX,DtD,MtD,YtD,DIVISION
0,20241129,IM16IDR0012001074391,Conven,2001074391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735,Maybank Giro,CFS-NONRB-BB,SIBALEC POWEL CABLE & ELECTRICAL SUPPLY,CA,IDR,NaN,NaN,1.25,6.923490e+08,6.923490e+08,369169.0,13903205.0,1.418543e+08,NaN
1,20241129,ST16IDR76730000076708941,Sharia,30000076708941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764,Maybank Deposito iB,CFS-NONRB-BB,"SUMMARECON AGUNG TBK, PT",TD,IDR,0000034089,NaN,2.50,1.250825e+08,1.250825e+08,0.0,0.0,0.000000e+00,NaN
2,20241129,IM16IDR0082008600017,Conven,2008600017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945,Maybank Giro,CFS-NONRB-RSME,TJOKRO BERSAUDARA,CA,IDR,NaN,NaN,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN
3,20241129,IM16IDR0202020901674,Conven,2020901674,JAKARTA 1,FATMAWATI,KCP AMPERA,G000458489,108120,Maybank Giro,CFS-NONRB-RSME,"WADHA ARTHA ABADI, PT",CA,IDR,NaN,NaN,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN
4,20241129,IM16USD1212121101114,Conven,2121101114,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G000473942,870324,Maybank Giro,CFS-NONRB-RSME,"BADE MAKMUR ORISSA, PT",CA,USD,0000019696,NaN,0.00,4.801098e+07,4.801098e+07,-75751.0,376031.0,5.106229e+05,NaN


In [68]:
#Buat Unique Key di paling lower level utk Daily Funding
currFin['Key'] = (currFin['GCIF_NO'].astype(str) + currFin['ACCT_NO'].astype(str) + 
                  currFin['PROD_TYPE'].astype(str) + currFin['CURR_CODE'].astype(str)) 

print(currFin.shape)
currFin = currFin.drop_duplicates('Key',keep='first')
print(currFin.shape)

(81872, 24)
(81872, 24)


In [69]:
# Tag TBL_BAL data
currFin['SOURCE'] = 'TBL_BAL'

# TBL_BAL Calc

In [70]:
currCalc = currFin.groupby(['BASE_DT', 'FLAG', 'SEGMENT', 'PROD_TYPE', 'DIVISION'], dropna=False)['BASE_AMT_FIX'].agg(BASE_AMT_FIX = 'sum')
currCalc.reset_index(inplace=True)

In [71]:
currCalc.head()

,BASE_DT,FLAG,SEGMENT,PROD_TYPE,DIVISION,BASE_AMT_FIX
0,20241129,Conven,CFS-NONRB-BB,CA,NaN,6.065247e+12
1,20241129,Conven,CFS-NONRB-BB,SA,NaN,1.846311e+10
2,20241129,Conven,CFS-NONRB-BB,TD,NaN,5.420203e+12
3,20241129,Conven,CFS-NONRB-MICRO,CA,NaN,5.725653e+08
4,20241129,Conven,CFS-NONRB-MICRO,SA,NaN,3.924678e+09


In [72]:
currCalc['SOURCE'] = 'TBL_BAL_SUMMARY'

In [73]:
print(currFin.shape)
currFin = currFin.append(currCalc, ignore_index=True)
print(currFin.shape)

(81872, 25)
(81921, 25)


# RMR Add

In [74]:
# Tag RMR data
RMR['SOURCE'] = 'RMR'

In [75]:
# Uncomment if updating RMR
# currFin = currFin.append(RMR, ignore_index=True)

In [76]:
RMR.head()

,BASE_DT,FLAG,SEGMENT,PROD_TYPE,BASE_AMT_FIX,Unnamed: 5,Unnamed: 6,SOURCE
0,20241031,Conven,CFS-NONRB-BB,CA,6.208786e+12,NaN,NaN,RMR
1,20241031,Conven,CFS-NONRB-BB,SA,6.493364e+10,NaN,NaN,RMR
2,20241031,Conven,CFS-NONRB-BB,TD,6.075005e+12,NaN,NaN,RMR
3,20241031,Sharia,CFS-NONRB-BB,CA,3.306730e+12,NaN,NaN,RMR
4,20241031,Sharia,CFS-NONRB-BB,SA,2.791680e+09,NaN,NaN,RMR


In [77]:
currFin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81921 entries, 0 to 81920
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BASE_DT       81921 non-null  int64  
 1   AGREE_ID      81872 non-null  object 
 2   FLAG          81921 non-null  object 
 3   ACCT_NO       81872 non-null  object 
 4   REGION        81871 non-null  object 
 5   AREA          81871 non-null  object 
 6   BRANCH        81871 non-null  object 
 7   GCIF_NO       81872 non-null  object 
 8   CIF_NO        81872 non-null  float64
 9   SUB_PROD_NM   81872 non-null  object 
 10  SEGMENT       81921 non-null  object 
 11  GCIF_NAME     81872 non-null  object 
 12  PROD_TYPE     81921 non-null  object 
 13  CURR_CODE     81872 non-null  object 
 14  NPK_SALES     78038 non-null  object 
 15  COLT          1279 non-null   float64
 16  RATE_DPK      81872 non-null  float64
 17  BASE_AMOUNT   81872 non-null  float64
 18  BASE_AMT_FIX  81921 non-nu

# LoB Mapping & Sorting

In [78]:
# Create Lob Mapping
lobMap = pd.DataFrame({'SEGMENT':["CFS-NONRB-MICRO", "CFS-NONRB-RSME", "CFS-NONRB-SME+", "CFS-NONRB-BB", "XXX-GB-CORP"],
                       'SEGMENT_FIX':["CFS-SMER & MICRO","CFS-SMER & MICRO", "CFS-SME+", "CFS-BB", "GB-CORP"]})

In [79]:
lobMap

,SEGMENT,SEGMENT_FIX
0,CFS-NONRB-MICRO,CFS-SMER & MICRO
1,CFS-NONRB-RSME,CFS-SMER & MICRO
2,CFS-NONRB-SME+,CFS-SME+
3,CFS-NONRB-BB,CFS-BB
4,XXX-GB-CORP,GB-CORP


In [80]:
# Join
print(currFin.shape)
currFin = currFin.merge(lobMap, on='SEGMENT', how='left')
print(currFin.shape)

(81921, 25)
(81921, 26)


In [81]:
currFin

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,COLT,RATE_DPK,BASE_AMOUNT,BASE_AMT_FIX,DtD,MtD,YtD,DIVISION,Key,SOURCE,SEGMENT_FIX
0,20241129,IM16IDR0012001074391,Conven,2001074391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735.0,Maybank Giro,CFS-NONRB-BB,SIBALEC POWEL CABLE & ELECTRICAL SUPPLY,CA,IDR,NaN,NaN,1.25,6.923490e+08,6.923490e+08,369169.0,13903205.0,1.418543e+08,NaN,G0004554102001074391CAIDR,TBL_BAL,CFS-BB
1,20241129,ST16IDR76730000076708941,Sharia,30000076708941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764.0,Maybank Deposito iB,CFS-NONRB-BB,"SUMMARECON AGUNG TBK, PT",TD,IDR,0000034089,NaN,2.50,1.250825e+08,1.250825e+08,0.0,0.0,0.000000e+00,NaN,G00045662430000076708941TDIDR,TBL_BAL,CFS-BB
2,20241129,IM16IDR0082008600017,Conven,2008600017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945.0,Maybank Giro,CFS-NONRB-RSME,TJOKRO BERSAUDARA,CA,IDR,NaN,NaN,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN,G0004581662008600017CAIDR,TBL_BAL,CFS-SMER & MICRO
3,20241129,IM16IDR0202020901674,Conven,2020901674,JAKARTA 1,FATMAWATI,KCP AMPERA,G000458489,108120.0,Maybank Giro,CFS-NONRB-RSME,"WADHA ARTHA ABADI, PT",CA,IDR,NaN,NaN,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN,G0004584892020901674CAIDR,TBL_BAL,CFS-SMER & MICRO
4,20241129,IM16USD1212121101114,Conven,2121101114,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G000473942,870324.0,Maybank Giro,CFS-NONRB-RSME,"BADE MAKMUR ORISSA, PT",CA,USD,0000019696,NaN,0.00,4.801098e+07,4.801098e+07,-75751.0,376031.0,5.106229e+05,NaN,G0004739422121101114CAUSD,TBL_BAL,CFS-SMER & MICRO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81916,20241129,NaN,Sharia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXX-GB-CORP,NaN,TD,NaN,NaN,NaN,NaN,NaN,5.802708e+10,NaN,NaN,NaN,BANK,NaN,TBL_BAL_SUMMARY,GB-CORP
81917,20241129,NaN,Sharia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXX-GB-CORP,NaN,TD,NaN,NaN,NaN,NaN,NaN,1.987884e+12,NaN,NaN,NaN,LLC & MNC,NaN,TBL_BAL_SUMMARY,GB-CORP
81918,20241129,NaN,Sharia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXX-GB-CORP,NaN,TD,NaN,NaN,NaN,NaN,NaN,6.911643e+11,NaN,NaN,NaN,NBFI,NaN,TBL_BAL_SUMMARY,GB-CORP
81919,20241129,NaN,Sharia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XXX-GB-CORP,NaN,TD,NaN,NaN,NaN,NaN,NaN,3.000000e+10,NaN,NaN,NaN,SOE,NaN,TBL_BAL_SUMMARY,GB-CORP


# Target Add

In [82]:
TARGET['SOURCE'] = 'TARGET'

In [83]:
# Uncomment if updating TARGET
# currFin = currFin.append(TARGET, ignore_index=True)

In [84]:
TARGET.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BASE_DT       24 non-null     int64  
 1   FLAG          24 non-null     object 
 2   SEGMENT_FIX   24 non-null     object 
 3   PROD_TYPE     24 non-null     object 
 4   BASE_AMT_FIX  24 non-null     float64
 5   Unnamed: 5    0 non-null      float64
 6   Unnamed: 6    0 non-null      float64
 7   Unnamed: 7    0 non-null      float64
 8   Unnamed: 8    0 non-null      float64
 9   SOURCE        24 non-null     object 
dtypes: float64(5), int64(1), object(4)
memory usage: 2.0+ KB


In [85]:
TARGET.head()

,BASE_DT,FLAG,SEGMENT_FIX,PROD_TYPE,BASE_AMT_FIX,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,SOURCE
0,20241110,Conven,CFS-BB,CA,1.055089e+13,NaN,NaN,NaN,NaN,TARGET
1,20241110,Conven,CFS-BB,SA,7.279970e+10,NaN,NaN,NaN,NaN,TARGET
2,20241110,Conven,CFS-BB,TD,6.558724e+12,NaN,NaN,NaN,NaN,TARGET
3,20241110,Sharia,CFS-BB,CA,0.000000e+00,NaN,NaN,NaN,NaN,TARGET
4,20241110,Sharia,CFS-BB,SA,0.000000e+00,NaN,NaN,NaN,NaN,TARGET


In [86]:
currFin['BASE_DT'] = currFin['BASE_DT'].astype(int)

In [87]:
print(currFin.shape)

(81921, 26)


In [88]:
currFin[currFin['GCIF_NO']=='G302105515']

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,COLT,RATE_DPK,BASE_AMOUNT,BASE_AMT_FIX,DtD,MtD,YtD,DIVISION,Key,SOURCE,SEGMENT_FIX
42555,20241129,IM16IDR1212121712442,Conven,2121712442,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G302105515,6038286.0,Maybank Giro,CFS-NONRB-RSME,IWK THAILAND LIMITED,CA,IDR,0000019696,NaN,0.75,2.531914e+08,2.531914e+08,-47530287.0,-74818713.0,156971189.0,NaN,G3021055152121712442CAIDR,TBL_BAL,CFS-SMER & MICRO


# COF MTD

In [89]:
mtd_tbl_bal.head(3)

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,RATE_DPK,BASE_AMOUNT,NPK_SALES,COLT
0,20241101,IM16IDR7082708000760,Sharia,2708000760,JAWA TENGAH,SEMARANG,KCS SYARIAH SEMARANG,G606861714,8010408,Maybank Giro iB,CFS-NONRB-BB,"KARYADEKA ALAM LESTARI, PT",CA,IDR,1.25,7.208821e+08,0000021619,NaN
2,20241101,IM16IDR1472147004151,Conven,2147004151,"JATIM, BALI & NUSRA",JATIM 4,KCP PERAK SURABAYA,G200734895,3953640,Maybank Giro,CFS-NONRB-RSME,"SARANA TEKNIK WIRATAMA, PT",CA,IDR,1.25,7.101538e+08,NaN,NaN
9,20241101,IM16IDR7802780000360,Sharia,2780000360,SUMATERA UTARA,MEDAN BARAT,KCS DIPONEGORO MEDAN,G809961287,13011618,Maybank Giro iB,CFS-NONRB-RSME,MAHKOTA PEMIMPIN MUDA PT,CA,IDR,1.50,1.651203e+09,0000037454,NaN


In [90]:
mtd_tbl_bal2 = mtd_tbl_bal.copy()

In [91]:
#Buat Unique Key di paling lower level
mtd_tbl_bal2['Key'] = (mtd_tbl_bal2['GCIF_NO'].astype(str) + mtd_tbl_bal2['ACCT_NO'].astype(str) + 
                       mtd_tbl_bal2['PROD_TYPE'].astype(str) + mtd_tbl_bal2['CURR_CODE'].astype(str)
                       ) 

In [92]:
# Fix BASE_AMOUNT, zeroing negative values
mtd_tbl_bal2[['BASE_AMT_FIX']] = mtd_tbl_bal2[['BASE_AMOUNT']]
mtd_tbl_bal2.loc[mtd_tbl_bal2["BASE_AMT_FIX"] < 0, "BASE_AMT_FIX"] = 0

In [93]:
mtd_tbl_bal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2240243 entries, 0 to 3348219
Data columns (total 18 columns):
 #   Column       Dtype  
---  ------       -----  
 0   BASE_DT      int64  
 1   AGREE_ID     object 
 2   FLAG         object 
 3   ACCT_NO      object 
 4   REGION       object 
 5   AREA         object 
 6   BRANCH       object 
 7   GCIF_NO      object 
 8   CIF_NO       object 
 9   SUB_PROD_NM  object 
 10  SEGMENT      object 
 11  GCIF_NAME    object 
 12  PROD_TYPE    object 
 13  CURR_CODE    object 
 14  RATE_DPK     float64
 15  BASE_AMOUNT  float64
 16  NPK_SALES    object 
 17  COLT         float64
dtypes: float64(3), int64(1), object(14)
memory usage: 324.7+ MB


In [94]:
# Calculate Rate %
# Calculate Amount Nominal dari Interest Expenses
mtd_tbl_bal2['RATE_DPK2'] = mtd_tbl_bal2['RATE_DPK']/100
mtd_tbl_bal2['INT_EXPENSE_AMT_YEARLY'] = mtd_tbl_bal2['RATE_DPK2']*mtd_tbl_bal2['BASE_AMT_FIX']

In [95]:
mtd_tbl_bal2.head(3)

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,RATE_DPK,BASE_AMOUNT,NPK_SALES,COLT,Key,BASE_AMT_FIX,RATE_DPK2,INT_EXPENSE_AMT_YEARLY
0,20241101,IM16IDR7082708000760,Sharia,2708000760,JAWA TENGAH,SEMARANG,KCS SYARIAH SEMARANG,G606861714,8010408,Maybank Giro iB,CFS-NONRB-BB,"KARYADEKA ALAM LESTARI, PT",CA,IDR,1.25,7.208821e+08,0000021619,NaN,G6068617142708000760CAIDR,7.208821e+08,0.0125,9.011026e+06
2,20241101,IM16IDR1472147004151,Conven,2147004151,"JATIM, BALI & NUSRA",JATIM 4,KCP PERAK SURABAYA,G200734895,3953640,Maybank Giro,CFS-NONRB-RSME,"SARANA TEKNIK WIRATAMA, PT",CA,IDR,1.25,7.101538e+08,NaN,NaN,G2007348952147004151CAIDR,7.101538e+08,0.0125,8.876922e+06
9,20241101,IM16IDR7802780000360,Sharia,2780000360,SUMATERA UTARA,MEDAN BARAT,KCS DIPONEGORO MEDAN,G809961287,13011618,Maybank Giro iB,CFS-NONRB-RSME,MAHKOTA PEMIMPIN MUDA PT,CA,IDR,1.50,1.651203e+09,0000037454,NaN,G8099612872780000360CAIDR,1.651203e+09,0.0150,2.476805e+07


In [96]:
mtd_tbl_bal3 = mtd_tbl_bal2[['BASE_DT','Key','GCIF_NO','ACCT_NO','PROD_TYPE','CURR_CODE','RATE_DPK2',
                             'BASE_AMT_FIX','INT_EXPENSE_AMT_YEARLY']]

In [97]:
print(mtd_tbl_bal3.shape)
mtd_tbl_bal3.head(5)

(2240243, 9)


,BASE_DT,Key,GCIF_NO,ACCT_NO,PROD_TYPE,CURR_CODE,RATE_DPK2,BASE_AMT_FIX,INT_EXPENSE_AMT_YEARLY
0,20241101,G6068617142708000760CAIDR,G606861714,2708000760,CA,IDR,0.0125,7.208821e+08,9.011026e+06
2,20241101,G2007348952147004151CAIDR,G200734895,2147004151,CA,IDR,0.0125,7.101538e+08,8.876922e+06
9,20241101,G8099612872780000360CAIDR,G809961287,2780000360,CA,IDR,0.0150,1.651203e+09,2.476805e+07
12,20241101,G30699049030000072600123TDIDR,G306990490,30000072600123,TD,IDR,0.0250,3.231676e+08,8.079189e+06
18,20241101,G1097371302126000017CACNY,G109737130,2126000017,CA,CNY,0.0000,0.000000e+00,0.000000e+00


In [98]:
mtd_tbl_bal3[mtd_tbl_bal3['GCIF_NO']=='G302105515']

,BASE_DT,Key,GCIF_NO,ACCT_NO,PROD_TYPE,CURR_CODE,RATE_DPK2,BASE_AMT_FIX,INT_EXPENSE_AMT_YEARLY
267706,20241101,G3021055152121712442CAIDR,G302105515,2121712442,CA,IDR,0.0075,3.279551e+08,2.459663e+06
395379,20241104,G3021055152121712442CAIDR,G302105515,2121712442,CA,IDR,0.0075,3.201851e+08,2.401388e+06
737556,20241105,G3021055152121712442CAIDR,G302105515,2121712442,CA,IDR,0.0075,2.194768e+08,1.646076e+06
1248747,20241106,G3021055152121712442CAIDR,G302105515,2121712442,CA,IDR,0.0075,2.194768e+08,1.646076e+06
1589266,20241107,G3021055152121712442CAIDR,G302105515,2121712442,CA,IDR,0.0075,2.194768e+08,1.646076e+06
1679028,20241108,G3021055152121712442CAIDR,G302105515,2121712442,CA,IDR,0.0075,2.194768e+08,1.646076e+06
1735299,20241111,G3021055152121712442CAIDR,G302105515,2121712442,CA,IDR,0.0075,2.194768e+08,1.646076e+06
1874770,20241112,G3021055152121712442CAIDR,G302105515,2121712442,CA,IDR,0.0075,2.194768e+08,1.646076e+06
2034989,20241113,G3021055152121712442CAIDR,G302105515,2121712442,CA,IDR,0.0075,2.194768e+08,1.646076e+06
2079183,20241114,G3021055152121712442CAIDR,G302105515,2121712442,CA,IDR,0.0075,2.194768e+08,1.646076e+06


In [99]:
pvt_mtd_bal = mtd_tbl_bal3.pivot_table(index=['Key'], columns=[], values=['BASE_AMT_FIX','INT_EXPENSE_AMT_YEARLY'], aggfunc='sum')
pvt_mtd_bal = pvt_mtd_bal.reset_index()
pvt_mtd_bal = pvt_mtd_bal.fillna(0)
#pvt_mtd_bal = pvt_mtd_bal.rename(columns = {"BASE_AMT_FIX":"BASE_AMT_FIX_AGR", "INT_EXPENSE_AMT_YEARLY": "AGG_INT_EXP"}, errors="raise")
pvt_mtd_bal = pvt_mtd_bal.rename(columns = {"BASE_AMT_FIX":"BASE_AMT_MTD", "INT_EXPENSE_AMT_YEARLY": "INT_EXPENSE_MTD"}, errors="raise")
pvt_mtd_bal

#"BASE_AMT_MTD", "INT_EXP_MTD"


,Key,BASE_AMT_MTD,INT_EXPENSE_MTD
0,G00002278000001736009246SAIDR,1.082262e+05,0.0
1,G0000227801736009246SAIDR,9.739184e+05,0.0
2,G00012962700001053042048SAIDR,0.000000e+00,0.0
3,G00012962700001590999900SAIDR,0.000000e+00,0.0
4,G0001296271053042048SAIDR,0.000000e+00,0.0
...,...,...,...
130667,G9104375402787001808CAIDR,0.000000e+00,0.0
130668,G91043770630000073902959TDIDR,3.945600e+08,9864000.0
130669,G91043770630000073902960TDIDR,3.945600e+08,9864000.0
130670,G91043770630000073902961TDIDR,1.972800e+08,4932000.0


In [100]:
pvt_mtd_bal[pvt_mtd_bal['Key']=='G3021055152121712442CAIDR']

,Key,BASE_AMT_MTD,INT_EXPENSE_MTD
40717,G3021055152121712442CAIDR,5.146668e+09,3.860001e+07


In [101]:
#Perhitungan COF MTD: INT_EXP_MTD/BASE_AMT_MTD
#INT_EXP_MTD = AGGREGATED amount dari daily interest expense, diakumulasi secara MTD
#BASE_AMT_MTD = AGGREGATED amount dari daily funding balance, diakumulasi secara MTD

pvt_mtd_bal['COF_MTD'] = pvt_mtd_bal["INT_EXPENSE_MTD"] / pvt_mtd_bal["BASE_AMT_MTD"]
pvt_mtd_bal = pvt_mtd_bal.fillna(0)
print(pvt_mtd_bal.shape)
pvt_mtd_bal

(130672, 4)


,Key,BASE_AMT_MTD,INT_EXPENSE_MTD,COF_MTD
0,G00002278000001736009246SAIDR,1.082262e+05,0.0,0.0000
1,G0000227801736009246SAIDR,9.739184e+05,0.0,0.0000
2,G00012962700001053042048SAIDR,0.000000e+00,0.0,0.0000
3,G00012962700001590999900SAIDR,0.000000e+00,0.0,0.0000
4,G0001296271053042048SAIDR,0.000000e+00,0.0,0.0000
...,...,...,...,...
130667,G9104375402787001808CAIDR,0.000000e+00,0.0,0.0000
130668,G91043770630000073902959TDIDR,3.945600e+08,9864000.0,0.0250
130669,G91043770630000073902960TDIDR,3.945600e+08,9864000.0,0.0250
130670,G91043770630000073902961TDIDR,1.972800e+08,4932000.0,0.0250


In [102]:
print(currFin.shape)
currFin.head(3)

(81921, 26)


,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,COLT,RATE_DPK,BASE_AMOUNT,BASE_AMT_FIX,DtD,MtD,YtD,DIVISION,Key,SOURCE,SEGMENT_FIX
0,20241129,IM16IDR0012001074391,Conven,2001074391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735.0,Maybank Giro,CFS-NONRB-BB,SIBALEC POWEL CABLE & ELECTRICAL SUPPLY,CA,IDR,NaN,NaN,1.25,6.923490e+08,6.923490e+08,369169.0,13903205.0,141854307.0,NaN,G0004554102001074391CAIDR,TBL_BAL,CFS-BB
1,20241129,ST16IDR76730000076708941,Sharia,30000076708941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764.0,Maybank Deposito iB,CFS-NONRB-BB,"SUMMARECON AGUNG TBK, PT",TD,IDR,0000034089,NaN,2.50,1.250825e+08,1.250825e+08,0.0,0.0,0.0,NaN,G00045662430000076708941TDIDR,TBL_BAL,CFS-BB
2,20241129,IM16IDR0082008600017,Conven,2008600017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945.0,Maybank Giro,CFS-NONRB-RSME,TJOKRO BERSAUDARA,CA,IDR,NaN,NaN,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,NaN,G0004581662008600017CAIDR,TBL_BAL,CFS-SMER & MICRO


In [103]:
# #Buat Unique Key di paling lower level utk Daily Funding
# currFin['Key'] = (currFin['GCIF_NO'].astype(str) + currFin['ACCT_NO'].astype(str) + 
#                   currFin['PROD_TYPE'].astype(str) + currFin['CURR_CODE'].astype(str)) 

# print(currFin.shape)
# currFin = currFin.drop_duplicates('Key',keep='first')
# print(currFin.shape)

In [104]:
# Merge MTD TBL BAL DATA into Daily Funding

print(currFin.shape)
currFin = pd.merge(currFin, pvt_mtd_bal[['Key','COF_MTD','BASE_AMT_MTD', 'INT_EXPENSE_MTD']], left_on='Key', right_on='Key',how='left')
print(currFin.shape)
currFin.head()

(81921, 26)
(81921, 29)


,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,COLT,RATE_DPK,BASE_AMOUNT,BASE_AMT_FIX,DtD,MtD,YtD,DIVISION,Key,SOURCE,SEGMENT_FIX,COF_MTD,BASE_AMT_MTD,INT_EXPENSE_MTD
0,20241129,IM16IDR0012001074391,Conven,2001074391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735.0,Maybank Giro,CFS-NONRB-BB,SIBALEC POWEL CABLE & ELECTRICAL SUPPLY,CA,IDR,NaN,NaN,1.25,6.923490e+08,6.923490e+08,369169.0,13903205.0,1.418543e+08,NaN,G0004554102001074391CAIDR,TBL_BAL,CFS-BB,0.012500,1.381279e+10,1.726598e+08
1,20241129,ST16IDR76730000076708941,Sharia,30000076708941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764.0,Maybank Deposito iB,CFS-NONRB-BB,"SUMMARECON AGUNG TBK, PT",TD,IDR,0000034089,NaN,2.50,1.250825e+08,1.250825e+08,0.0,0.0,0.000000e+00,NaN,G00045662430000076708941TDIDR,TBL_BAL,CFS-BB,0.041625,2.501649e+09,1.041312e+08
2,20241129,IM16IDR0082008600017,Conven,2008600017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945.0,Maybank Giro,CFS-NONRB-RSME,TJOKRO BERSAUDARA,CA,IDR,NaN,NaN,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN,G0004581662008600017CAIDR,TBL_BAL,CFS-SMER & MICRO,0.000000,0.000000e+00,0.000000e+00
3,20241129,IM16IDR0202020901674,Conven,2020901674,JAKARTA 1,FATMAWATI,KCP AMPERA,G000458489,108120.0,Maybank Giro,CFS-NONRB-RSME,"WADHA ARTHA ABADI, PT",CA,IDR,NaN,NaN,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN,G0004584892020901674CAIDR,TBL_BAL,CFS-SMER & MICRO,0.000000,0.000000e+00,0.000000e+00
4,20241129,IM16USD1212121101114,Conven,2121101114,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G000473942,870324.0,Maybank Giro,CFS-NONRB-RSME,"BADE MAKMUR ORISSA, PT",CA,USD,0000019696,NaN,0.00,4.801098e+07,4.801098e+07,-75751.0,376031.0,5.106229e+05,NaN,G0004739422121101114CAUSD,TBL_BAL,CFS-SMER & MICRO,0.000000,9.582426e+08,0.000000e+00


In [105]:
currFin[currFin['GCIF_NO']=='G302105515']

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,COLT,RATE_DPK,BASE_AMOUNT,BASE_AMT_FIX,DtD,MtD,YtD,DIVISION,Key,SOURCE,SEGMENT_FIX,COF_MTD,BASE_AMT_MTD,INT_EXPENSE_MTD
42555,20241129,IM16IDR1212121712442,Conven,2121712442,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G302105515,6038286.0,Maybank Giro,CFS-NONRB-RSME,IWK THAILAND LIMITED,CA,IDR,0000019696,NaN,0.75,2.531914e+08,2.531914e+08,-47530287.0,-74818713.0,156971189.0,NaN,G3021055152121712442CAIDR,TBL_BAL,CFS-SMER & MICRO,0.0075,5.146668e+09,3.860001e+07


In [106]:
currFin = currFin.drop('Key', axis='columns')
print(currFin.shape)
currFin.head()

(81921, 28)


,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,COLT,RATE_DPK,BASE_AMOUNT,BASE_AMT_FIX,DtD,MtD,YtD,DIVISION,SOURCE,SEGMENT_FIX,COF_MTD,BASE_AMT_MTD,INT_EXPENSE_MTD
0,20241129,IM16IDR0012001074391,Conven,2001074391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735.0,Maybank Giro,CFS-NONRB-BB,SIBALEC POWEL CABLE & ELECTRICAL SUPPLY,CA,IDR,NaN,NaN,1.25,6.923490e+08,6.923490e+08,369169.0,13903205.0,1.418543e+08,NaN,TBL_BAL,CFS-BB,0.012500,1.381279e+10,1.726598e+08
1,20241129,ST16IDR76730000076708941,Sharia,30000076708941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764.0,Maybank Deposito iB,CFS-NONRB-BB,"SUMMARECON AGUNG TBK, PT",TD,IDR,0000034089,NaN,2.50,1.250825e+08,1.250825e+08,0.0,0.0,0.000000e+00,NaN,TBL_BAL,CFS-BB,0.041625,2.501649e+09,1.041312e+08
2,20241129,IM16IDR0082008600017,Conven,2008600017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945.0,Maybank Giro,CFS-NONRB-RSME,TJOKRO BERSAUDARA,CA,IDR,NaN,NaN,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN,TBL_BAL,CFS-SMER & MICRO,0.000000,0.000000e+00,0.000000e+00
3,20241129,IM16IDR0202020901674,Conven,2020901674,JAKARTA 1,FATMAWATI,KCP AMPERA,G000458489,108120.0,Maybank Giro,CFS-NONRB-RSME,"WADHA ARTHA ABADI, PT",CA,IDR,NaN,NaN,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN,TBL_BAL,CFS-SMER & MICRO,0.000000,0.000000e+00,0.000000e+00
4,20241129,IM16USD1212121101114,Conven,2121101114,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G000473942,870324.0,Maybank Giro,CFS-NONRB-RSME,"BADE MAKMUR ORISSA, PT",CA,USD,0000019696,NaN,0.00,4.801098e+07,4.801098e+07,-75751.0,376031.0,5.106229e+05,NaN,TBL_BAL,CFS-SMER & MICRO,0.000000,9.582426e+08,0.000000e+00


# High Cost Funding

In [107]:
currFin['RATE_DPK2'] = currFin['RATE_DPK']/100
currFin['INT_EXPENSE_AMT_YEARLY'] = currFin['RATE_DPK2']*currFin['BASE_AMT_FIX']
#currFin['INT_EXPENSE_MTD'] = currFin['COF_MTD']*currFin['BASE_AMT_FIX']

In [108]:
# # Flagging High Cost Funding CA
# # PRD = CA
# # RATE IDR > 3%, USD >1%
# # NON BTB (COLT != 1)

# def ca_flag(cof):
#     #1
#     if (cof['PROD_TYPE'] == 'CA') and (cof['CURR_CODE'] == 'IDR') and (cof['RATE_DPK2'] <= 0.015) and (cof['COLT'] != 1) :
#         return 'NORMAL COF (≤ 1.5%)'
#     if (cof['PROD_TYPE'] == 'CA') and (cof['CURR_CODE'] == 'IDR') and ((cof['RATE_DPK2'] > 0.015) and (cof['RATE_DPK2'] <= 0.03))and (cof['COLT'] != 1) :
#         return 'MEDIUM COF (1.5 - 3.0%)' 
#     if (cof['PROD_TYPE'] == 'CA') and (cof['CURR_CODE'] == 'IDR') and (cof['RATE_DPK2'] > 0.03) and (cof['COLT'] != 1) :
#         return 'HIGH COF (> 3.0%)' 
#     #2
#     if (cof['PROD_TYPE'] == 'CA') and (cof['CURR_CODE'] == 'USD') and (cof['RATE_DPK2'] <= 0.0025) and (cof['COLT'] != 1) :
#         return 'NORMAL COF (≤ 1.5%)'
#     if (cof['PROD_TYPE'] == 'CA') and (cof['CURR_CODE'] == 'USD') and ((cof['RATE_DPK2'] > 0.0025) and (cof['RATE_DPK2'] <= 0.01))and (cof['COLT'] != 1) :
#         return 'MEDIUM COF (1.5 - 3.0%)' 
#     if (cof['PROD_TYPE'] == 'CA') and (cof['CURR_CODE'] == 'USD') and (cof['RATE_DPK2'] > 0.01) and (cof['COLT'] != 1) :
#         return 'HIGH COF (> 3.0%)' 
#     else:
#         return ''


# currFin['HIGH_COF_FLAG'] = currFin.apply(ca_flag, axis=1)
# print(currFin.shape)
# currFin['HIGH_COF_FLAG'].value_counts(dropna=False)
    

In [109]:
# Flagging High Cost Funding CA
# PRD = CA
# RATE IDR > 3%, USD >1%
# NON BTB (COLT != 1)

def ca_flag(cof):
    #1
    if (cof['PROD_TYPE'] == 'CA') and (cof['CURR_CODE'] == 'IDR') and (cof['COF_MTD'] <= 0.015) and (cof['COLT'] != 1) :
        return 'NORMAL COF (IDR ≤1.5% & USD ≤0.25%)'
    if (cof['PROD_TYPE'] == 'CA') and (cof['CURR_CODE'] == 'IDR') and ((cof['COF_MTD'] > 0.015) and (cof['COF_MTD'] <= 0.03))and (cof['COLT'] != 1) :
        return 'MEDIUM COF (IDR 1.5-3.0% & USD 0.25-1.00%)' 
    if (cof['PROD_TYPE'] == 'CA') and (cof['CURR_CODE'] == 'IDR') and (cof['COF_MTD'] > 0.03) and (cof['COLT'] != 1) :
        return 'HIGH COF (IDR >3.0% & USD >1.0%)' 
    #2
    if (cof['PROD_TYPE'] == 'CA') and (cof['CURR_CODE'] == 'USD') and (cof['COF_MTD'] <= 0.0025) and (cof['COLT'] != 1) :
        return 'NORMAL COF (IDR ≤1.5% & USD ≤0.25%)'
    if (cof['PROD_TYPE'] == 'CA') and (cof['CURR_CODE'] == 'USD') and ((cof['COF_MTD'] > 0.0025) and (cof['COF_MTD'] <= 0.01))and (cof['COLT'] != 1) :
        return 'MEDIUM COF (IDR 1.5-3.0% & USD 0.25-1.00%)' 
    if (cof['PROD_TYPE'] == 'CA') and (cof['CURR_CODE'] == 'USD') and (cof['COF_MTD'] > 0.01) and (cof['COLT'] != 1) :
        return 'HIGH COF (IDR >3.0% & USD >1.0%)' 
    else:
        return ''


currFin['HIGH_COF_FLAG'] = currFin.apply(ca_flag, axis=1)
print(currFin.shape)
currFin['HIGH_COF_FLAG'].value_counts(dropna=False)
    

(81921, 31)


NORMAL COF (IDR ≤1.5% & USD ≤0.25%)           40378
                                              40155
MEDIUM COF (IDR 1.5-3.0% & USD 0.25-1.00%)      756
HIGH COF (IDR >3.0% & USD >1.0%)                632
Name: HIGH_COF_FLAG, dtype: int64

In [110]:
print(currFin.shape)
currFin.head()

(81921, 31)


,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,...,RATE_DPK,BASE_AMOUNT,BASE_AMT_FIX,DtD,MtD,YtD,DIVISION,SOURCE,SEGMENT_FIX,COF_MTD,BASE_AMT_MTD,INT_EXPENSE_MTD,RATE_DPK2,INT_EXPENSE_AMT_YEARLY,HIGH_COF_FLAG
0,20241129,IM16IDR0012001074391,Conven,2001074391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735.0,Maybank Giro,CFS-NONRB-BB,SIBALEC POWEL CABLE & ELECTRICAL SUPPLY,CA,IDR,NaN,...,1.25,6.923490e+08,6.923490e+08,369169.0,13903205.0,1.418543e+08,NaN,TBL_BAL,CFS-BB,0.012500,1.381279e+10,1.726598e+08,0.0125,8.654362e+06,NORMAL COF (IDR ≤1.5% & USD ≤0.25%)
1,20241129,ST16IDR76730000076708941,Sharia,30000076708941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764.0,Maybank Deposito iB,CFS-NONRB-BB,"SUMMARECON AGUNG TBK, PT",TD,IDR,0000034089,...,2.50,1.250825e+08,1.250825e+08,0.0,0.0,0.000000e+00,NaN,TBL_BAL,CFS-BB,0.041625,2.501649e+09,1.041312e+08,0.0250,3.127062e+06,
2,20241129,IM16IDR0082008600017,Conven,2008600017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945.0,Maybank Giro,CFS-NONRB-RSME,TJOKRO BERSAUDARA,CA,IDR,NaN,...,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN,TBL_BAL,CFS-SMER & MICRO,0.000000,0.000000e+00,0.000000e+00,0.0000,0.000000e+00,NORMAL COF (IDR ≤1.5% & USD ≤0.25%)
3,20241129,IM16IDR0202020901674,Conven,2020901674,JAKARTA 1,FATMAWATI,KCP AMPERA,G000458489,108120.0,Maybank Giro,CFS-NONRB-RSME,"WADHA ARTHA ABADI, PT",CA,IDR,NaN,...,0.00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN,TBL_BAL,CFS-SMER & MICRO,0.000000,0.000000e+00,0.000000e+00,0.0000,0.000000e+00,NORMAL COF (IDR ≤1.5% & USD ≤0.25%)
4,20241129,IM16USD1212121101114,Conven,2121101114,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G000473942,870324.0,Maybank Giro,CFS-NONRB-RSME,"BADE MAKMUR ORISSA, PT",CA,USD,0000019696,...,0.00,4.801098e+07,4.801098e+07,-75751.0,376031.0,5.106229e+05,NaN,TBL_BAL,CFS-SMER & MICRO,0.000000,9.582426e+08,0.000000e+00,0.0000,0.000000e+00,NORMAL COF (IDR ≤1.5% & USD ≤0.25%)


In [111]:
currFin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81921 entries, 0 to 81920
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BASE_DT                 81921 non-null  int32  
 1   AGREE_ID                81872 non-null  object 
 2   FLAG                    81921 non-null  object 
 3   ACCT_NO                 81872 non-null  object 
 4   REGION                  81871 non-null  object 
 5   AREA                    81871 non-null  object 
 6   BRANCH                  81871 non-null  object 
 7   GCIF_NO                 81872 non-null  object 
 8   CIF_NO                  81872 non-null  float64
 9   SUB_PROD_NM             81872 non-null  object 
 10  SEGMENT                 81921 non-null  object 
 11  GCIF_NAME               81872 non-null  object 
 12  PROD_TYPE               81921 non-null  object 
 13  CURR_CODE               81872 non-null  object 
 14  NPK_SALES               78038 non-null

# MASKING

In [112]:
acct_fix = []
agr_fix = []
for i in list(currFin.index):
    agree = str(currFin.loc[i,'AGREE_ID'])
    a = agree[:-7] + "XXXX" + agree[-3:]
    acct = str(currFin.loc[i, "ACCT_NO"])
    b = acct[:-7] + "XXXX" + acct[-3:]
    agr_fix.append(a)
    acct_fix.append(b)
currFin['AGREE_ID'] = agr_fix
currFin['ACCT_NO'] = acct_fix

In [113]:
# Create LoB sorting
lobSort = pd.DataFrame({'LOB_SORT':[1, 2, 3, 4],
                       'SEGMENT_FIX':["CFS-SMER & MICRO", "CFS-SME+", "CFS-BB", "GB-CORP"]})

In [114]:
# Join
print(currFin.shape)
currFin = currFin.merge(lobSort, on='SEGMENT_FIX', how='left')
print(currFin.shape)
currFin = currFin.merge(FFlag, left_on='GCIF_NO', right_on='GCIF', how='left')
print(currFin.shape)

(81921, 31)
(81921, 32)
(81921, 34)


In [115]:
print(currFin.shape)
currFin.head()

(81921, 34)


,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,...,DtD,MtD,YtD,DIVISION,SOURCE,SEGMENT_FIX,COF_MTD,BASE_AMT_MTD,INT_EXPENSE_MTD,RATE_DPK2,INT_EXPENSE_AMT_YEARLY,HIGH_COF_FLAG,LOB_SORT,GCIF,FINAL_FLAG
0,20241129,IM16IDR001200XXXX391,Conven,200XXXX391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735.0,Maybank Giro,CFS-NONRB-BB,SIBALEC POWEL CABLE & ELECTRICAL SUPPLY,CA,IDR,NaN,...,369169.0,13903205.0,1.418543e+08,NaN,TBL_BAL,CFS-BB,0.012500,1.381279e+10,1.726598e+08,0.0125,8.654362e+06,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),3,G000455410,LOW
1,20241129,ST16IDR7673000007XXXX941,Sharia,3000007XXXX941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764.0,Maybank Deposito iB,CFS-NONRB-BB,"SUMMARECON AGUNG TBK, PT",TD,IDR,0000034089,...,0.0,0.0,0.000000e+00,NaN,TBL_BAL,CFS-BB,0.041625,2.501649e+09,1.041312e+08,0.0250,3.127062e+06,,3,G000456624,LOW
2,20241129,IM16IDR008200XXXX017,Conven,200XXXX017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945.0,Maybank Giro,CFS-NONRB-RSME,TJOKRO BERSAUDARA,CA,IDR,NaN,...,0.0,0.0,0.000000e+00,NaN,TBL_BAL,CFS-SMER & MICRO,0.000000,0.000000e+00,0.000000e+00,0.0000,0.000000e+00,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),1,NaN,NaN
3,20241129,IM16IDR020202XXXX674,Conven,202XXXX674,JAKARTA 1,FATMAWATI,KCP AMPERA,G000458489,108120.0,Maybank Giro,CFS-NONRB-RSME,"WADHA ARTHA ABADI, PT",CA,IDR,NaN,...,0.0,0.0,0.000000e+00,NaN,TBL_BAL,CFS-SMER & MICRO,0.000000,0.000000e+00,0.000000e+00,0.0000,0.000000e+00,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),1,NaN,NaN
4,20241129,IM16USD121212XXXX114,Conven,212XXXX114,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G000473942,870324.0,Maybank Giro,CFS-NONRB-RSME,"BADE MAKMUR ORISSA, PT",CA,USD,0000019696,...,-75751.0,376031.0,5.106229e+05,NaN,TBL_BAL,CFS-SMER & MICRO,0.000000,9.582426e+08,0.000000e+00,0.0000,0.000000e+00,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),1,G000473942,LOW


In [116]:
currFin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81921 entries, 0 to 81920
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BASE_DT                 81921 non-null  int32  
 1   AGREE_ID                81921 non-null  object 
 2   FLAG                    81921 non-null  object 
 3   ACCT_NO                 81921 non-null  object 
 4   REGION                  81871 non-null  object 
 5   AREA                    81871 non-null  object 
 6   BRANCH                  81871 non-null  object 
 7   GCIF_NO                 81872 non-null  object 
 8   CIF_NO                  81872 non-null  float64
 9   SUB_PROD_NM             81872 non-null  object 
 10  SEGMENT                 81921 non-null  object 
 11  GCIF_NAME               81872 non-null  object 
 12  PROD_TYPE               81921 non-null  object 
 13  CURR_CODE               81872 non-null  object 
 14  NPK_SALES               78038 non-null

In [117]:
def masking_name(param):

    if param not in ["nan","NaN"]:
        return  ' '.join([item.replace(item[1:-1], "*"*len(item[1:-1])) if len(item)>2 and item[-1]!="," and item[-1]!="." else (item.replace(item[1:-2], "*"*len(item[1:-2])) if len(item)>2 and (item[-1]==",") else (item if len(item)>2 and (item[-1]==".") else item.replace(item[1:-2], "*"*len(item[1:-2])))) for item in param.split()])
    else:
        return param

In [118]:
currFin["GCIF_NAME"] = currFin["GCIF_NAME"].astype(str)
currFin["GCIF_NAME"] = currFin["GCIF_NAME"].apply(masking_name) 

In [119]:
currFin.columns

Index(['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH',
       'GCIF_NO', 'CIF_NO', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE',
       'CURR_CODE', 'NPK_SALES', 'COLT', 'RATE_DPK', 'BASE_AMOUNT',
       'BASE_AMT_FIX', 'DtD', 'MtD', 'YtD', 'DIVISION', 'SOURCE',
       'SEGMENT_FIX', 'COF_MTD', 'BASE_AMT_MTD', 'INT_EXPENSE_MTD',
       'RATE_DPK2', 'INT_EXPENSE_AMT_YEARLY', 'HIGH_COF_FLAG', 'LOB_SORT',
       'GCIF', 'FINAL_FLAG'],
      dtype='object')

In [120]:
currFin = currFin[['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH',
       'GCIF_NO', 'CIF_NO', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE',
       'CURR_CODE', 'NPK_SALES', 'COLT', 'RATE_DPK', 'BASE_AMOUNT',
       'BASE_AMT_FIX', 'DtD', 'MtD', 'YtD', 'DIVISION', 'SOURCE','BASE_AMT_MTD',
       'SEGMENT_FIX', 'COF_MTD', 'RATE_DPK2', 'INT_EXPENSE_AMT_YEARLY',
       'INT_EXPENSE_MTD','HIGH_COF_FLAG', 'LOB_SORT']]



In [121]:
currFin

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,...,BASE_AMOUNT,BASE_AMT_FIX,DtD,MtD,YtD,DIVISION,SOURCE,BASE_AMT_MTD,SEGMENT_FIX,COF_MTD,RATE_DPK2,INT_EXPENSE_AMT_YEARLY,INT_EXPENSE_MTD,HIGH_COF_FLAG,LOB_SORT
0,20241129,IM16IDR001200XXXX391,Conven,200XXXX391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735.0,Maybank Giro,CFS-NONRB-BB,S*****C P***L C***E & E********L S****Y,CA,IDR,NaN,...,6.923490e+08,6.923490e+08,369169.0,13903205.0,1.418543e+08,NaN,TBL_BAL,1.381279e+10,CFS-BB,0.012500,0.0125,8.654362e+06,1.726598e+08,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),3
1,20241129,ST16IDR7673000007XXXX941,Sharia,3000007XXXX941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764.0,Maybank Deposito iB,CFS-NONRB-BB,"S********N A***G T*K, PT",TD,IDR,0000034089,...,1.250825e+08,1.250825e+08,0.0,0.0,0.000000e+00,NaN,TBL_BAL,2.501649e+09,CFS-BB,0.041625,0.0250,3.127062e+06,1.041312e+08,,3
2,20241129,IM16IDR008200XXXX017,Conven,200XXXX017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945.0,Maybank Giro,CFS-NONRB-RSME,T****O B********A,CA,IDR,NaN,...,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN,TBL_BAL,0.000000e+00,CFS-SMER & MICRO,0.000000,0.0000,0.000000e+00,0.000000e+00,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),1
3,20241129,IM16IDR020202XXXX674,Conven,202XXXX674,JAKARTA 1,FATMAWATI,KCP AMPERA,G000458489,108120.0,Maybank Giro,CFS-NONRB-RSME,"W***A A***A A***I, PT",CA,IDR,NaN,...,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN,TBL_BAL,0.000000e+00,CFS-SMER & MICRO,0.000000,0.0000,0.000000e+00,0.000000e+00,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),1
4,20241129,IM16USD121212XXXX114,Conven,212XXXX114,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G000473942,870324.0,Maybank Giro,CFS-NONRB-RSME,"B**E M****R O****A, PT",CA,USD,0000019696,...,4.801098e+07,4.801098e+07,-75751.0,376031.0,5.106229e+05,NaN,TBL_BAL,9.582426e+08,CFS-SMER & MICRO,0.000000,0.0000,0.000000e+00,0.000000e+00,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81916,20241129,XXXXnan,Sharia,XXXXnan,NaN,NaN,NaN,NaN,NaN,NaN,XXX-GB-CORP,nan,TD,NaN,NaN,...,NaN,5.802708e+10,NaN,NaN,NaN,BANK,TBL_BAL_SUMMARY,NaN,GB-CORP,NaN,NaN,NaN,NaN,,4
81917,20241129,XXXXnan,Sharia,XXXXnan,NaN,NaN,NaN,NaN,NaN,NaN,XXX-GB-CORP,nan,TD,NaN,NaN,...,NaN,1.987884e+12,NaN,NaN,NaN,LLC & MNC,TBL_BAL_SUMMARY,NaN,GB-CORP,NaN,NaN,NaN,NaN,,4
81918,20241129,XXXXnan,Sharia,XXXXnan,NaN,NaN,NaN,NaN,NaN,NaN,XXX-GB-CORP,nan,TD,NaN,NaN,...,NaN,6.911643e+11,NaN,NaN,NaN,NBFI,TBL_BAL_SUMMARY,NaN,GB-CORP,NaN,NaN,NaN,NaN,,4
81919,20241129,XXXXnan,Sharia,XXXXnan,NaN,NaN,NaN,NaN,NaN,NaN,XXX-GB-CORP,nan,TD,NaN,NaN,...,NaN,3.000000e+10,NaN,NaN,NaN,SOE,TBL_BAL_SUMMARY,NaN,GB-CORP,NaN,NaN,NaN,NaN,,4


In [122]:
currFin.drop_duplicates(inplace=True)

In [123]:
currFin.head()

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,...,BASE_AMOUNT,BASE_AMT_FIX,DtD,MtD,YtD,DIVISION,SOURCE,BASE_AMT_MTD,SEGMENT_FIX,COF_MTD,RATE_DPK2,INT_EXPENSE_AMT_YEARLY,INT_EXPENSE_MTD,HIGH_COF_FLAG,LOB_SORT
0,20241129,IM16IDR001200XXXX391,Conven,200XXXX391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735.0,Maybank Giro,CFS-NONRB-BB,S*****C P***L C***E & E********L S****Y,CA,IDR,NaN,...,6.923490e+08,6.923490e+08,369169.0,13903205.0,1.418543e+08,NaN,TBL_BAL,1.381279e+10,CFS-BB,0.012500,0.0125,8.654362e+06,1.726598e+08,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),3
1,20241129,ST16IDR7673000007XXXX941,Sharia,3000007XXXX941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764.0,Maybank Deposito iB,CFS-NONRB-BB,"S********N A***G T*K, PT",TD,IDR,0000034089,...,1.250825e+08,1.250825e+08,0.0,0.0,0.000000e+00,NaN,TBL_BAL,2.501649e+09,CFS-BB,0.041625,0.0250,3.127062e+06,1.041312e+08,,3
2,20241129,IM16IDR008200XXXX017,Conven,200XXXX017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945.0,Maybank Giro,CFS-NONRB-RSME,T****O B********A,CA,IDR,NaN,...,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN,TBL_BAL,0.000000e+00,CFS-SMER & MICRO,0.000000,0.0000,0.000000e+00,0.000000e+00,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),1
3,20241129,IM16IDR020202XXXX674,Conven,202XXXX674,JAKARTA 1,FATMAWATI,KCP AMPERA,G000458489,108120.0,Maybank Giro,CFS-NONRB-RSME,"W***A A***A A***I, PT",CA,IDR,NaN,...,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,NaN,TBL_BAL,0.000000e+00,CFS-SMER & MICRO,0.000000,0.0000,0.000000e+00,0.000000e+00,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),1
4,20241129,IM16USD121212XXXX114,Conven,212XXXX114,JAKARTA 1,MT HARYONO,KCI MT HARYONO,G000473942,870324.0,Maybank Giro,CFS-NONRB-RSME,"B**E M****R O****A, PT",CA,USD,0000019696,...,4.801098e+07,4.801098e+07,-75751.0,376031.0,5.106229e+05,NaN,TBL_BAL,9.582426e+08,CFS-SMER & MICRO,0.000000,0.0000,0.000000e+00,0.000000e+00,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),1


# Save Result

In [124]:
# print(currFin.shape)
# currFin=currFin[currFin['DIVISION']=='BANK']
# print(currFin.shape)
# currFin=currFin[currFin['SOURCE']!='TBL_BAL_SUMMARY']
# print(currFin.shape)
# currFin=currFin[~currFin['GCIF_NO'].isin(['G205125842','G208395780','G805691623','G909262974','G709022446','G709825244','G206551194','G309699478','G709551923'])]
# print(currFin.shape)

# currFin

In [125]:
# Save result


# ## Function to convert 'dueDate' column into date format
# def convert_date(param):
#     try:
#         return datetime.strptime(param, '%Y%m%d').strftime('%Y-%m-%d')
#     except:
#         pass
# currFin['BASE_DT'] = currFin['BASE_DT'].astype(str)
# currFin['Month'] = currFin['BASE_DT'].apply(convert_date)


currFin['Month'] = current_d2
currFin['CIF_NO'] = currFin['CIF_NO'].astype(str)
currFin['BASE_DT'] = currFin['BASE_DT'].astype(int)


# currFin2 = currFin[['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH',
#        'GCIF_NO', 'CIF_NO', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE',
#        'CURR_CODE', 'NPK_SALES', 'COLT', 'RATE_DPK', 'BASE_AMOUNT',
#        'BASE_AMT_FIX', 'DtD', 'MtD', 'YtD', 'DIVISION', 'SOURCE',
#        'SEGMENT_FIX', 'COF_MTD', 'RATE_DPK2', 'INT_EXPENSE_AMT_YEARLY',
#        'INT_EXPENSE_MTD', 'HIGH_COF_FLAG', 'LOB_SORT','Month']]


# currFin.to_excel(finalName[:-4]+"_test.xlsx", index=False)

In [126]:
currFin2 = currFin[['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH',
       'GCIF_NO', 'CIF_NO', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE',
       'CURR_CODE', 'NPK_SALES', 'COLT', 'RATE_DPK', 'BASE_AMOUNT',
       'BASE_AMT_FIX', 'DtD', 'MtD', 'YtD', 'DIVISION', 'SOURCE',
       'SEGMENT_FIX', 'COF_MTD', 'RATE_DPK2', 'INT_EXPENSE_AMT_YEARLY','BASE_AMT_MTD',
       'INT_EXPENSE_MTD', 'HIGH_COF_FLAG', 'LOB_SORT','Month']]




In [127]:
# print(currFin2.shape)
# currFin2 = currFin2[currFin2['GCIF_NO'].isin(bank_list2)]
# print(currFin2.shape)

In [128]:
currFin2.to_csv(finalName, index=False)

In [129]:
currFin.columns

Index(['BASE_DT', 'AGREE_ID', 'FLAG', 'ACCT_NO', 'REGION', 'AREA', 'BRANCH',
       'GCIF_NO', 'CIF_NO', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE',
       'CURR_CODE', 'NPK_SALES', 'COLT', 'RATE_DPK', 'BASE_AMOUNT',
       'BASE_AMT_FIX', 'DtD', 'MtD', 'YtD', 'DIVISION', 'SOURCE',
       'BASE_AMT_MTD', 'SEGMENT_FIX', 'COF_MTD', 'RATE_DPK2',
       'INT_EXPENSE_AMT_YEARLY', 'INT_EXPENSE_MTD', 'HIGH_COF_FLAG',
       'LOB_SORT', 'Month'],
      dtype='object')

In [130]:
currFin.head(3)

,BASE_DT,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,SUB_PROD_NM,SEGMENT,GCIF_NAME,PROD_TYPE,CURR_CODE,NPK_SALES,...,BASE_AMT_FIX,DtD,MtD,YtD,DIVISION,SOURCE,BASE_AMT_MTD,SEGMENT_FIX,COF_MTD,RATE_DPK2,INT_EXPENSE_AMT_YEARLY,INT_EXPENSE_MTD,HIGH_COF_FLAG,LOB_SORT,Month
0,20241129,IM16IDR001200XXXX391,Conven,200XXXX391,JAKARTA 2,EKAJIWA,KCI EKAJIWA,G000455410,8735.0,Maybank Giro,CFS-NONRB-BB,S*****C P***L C***E & E********L S****Y,CA,IDR,NaN,...,6.923490e+08,369169.0,13903205.0,141854307.0,NaN,TBL_BAL,1.381279e+10,CFS-BB,0.012500,0.0125,8.654362e+06,1.726598e+08,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),3,2024-11-29
1,20241129,ST16IDR7673000007XXXX941,Sharia,3000007XXXX941,KPR CENTER,KPR CENTER,KPR CENTER,G000456624,44764.0,Maybank Deposito iB,CFS-NONRB-BB,"S********N A***G T*K, PT",TD,IDR,0000034089,...,1.250825e+08,0.0,0.0,0.0,NaN,TBL_BAL,2.501649e+09,CFS-BB,0.041625,0.0250,3.127062e+06,1.041312e+08,,3,2024-11-29
2,20241129,IM16IDR008200XXXX017,Conven,200XXXX017,JAKARTA 1,FATMAWATI,KCI FATMAWATI,G000458166,100945.0,Maybank Giro,CFS-NONRB-RSME,T****O B********A,CA,IDR,NaN,...,0.000000e+00,0.0,0.0,0.0,NaN,TBL_BAL,0.000000e+00,CFS-SMER & MICRO,0.000000,0.0000,0.000000e+00,0.000000e+00,NORMAL COF (IDR ≤1.5% & USD ≤0.25%),1,2024-11-29


In [131]:
currFin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81918 entries, 0 to 81920
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BASE_DT                 81918 non-null  int32  
 1   AGREE_ID                81918 non-null  object 
 2   FLAG                    81918 non-null  object 
 3   ACCT_NO                 81918 non-null  object 
 4   REGION                  81868 non-null  object 
 5   AREA                    81868 non-null  object 
 6   BRANCH                  81868 non-null  object 
 7   GCIF_NO                 81869 non-null  object 
 8   CIF_NO                  81918 non-null  object 
 9   SUB_PROD_NM             81869 non-null  object 
 10  SEGMENT                 81918 non-null  object 
 11  GCIF_NAME               81918 non-null  object 
 12  PROD_TYPE               81918 non-null  object 
 13  CURR_CODE               81869 non-null  object 
 14  NPK_SALES               78035 non-null